# Import

In [1]:
import numpy as np
print(f'numpy version = {np.__version__}')

import tensorflow as tf
print(f'tensorflow version = {tf.__version__}')

numpy version = 1.21.5
tensorflow version = 2.5.0


In [2]:
import os
import cv2
import csv
import sys
import math
import operator
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from tableone import TableOne
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score, auc
from mlxtend.plotting import plot_confusion_matrix

# Global Variables

In [3]:
n_FOLD = 5
n_SUBTASK = 4

remove_dementia_pid_list = [22, 33, 57, 58, 64, 66, 67]
remove_dementia_pid_list_str = ['22', '33', '57', '58', '64', '66', '67']

random_seed_list = [10, 20, 42, 64, 100, 123, 200, 456, 1001, 12321]

w_result_randomseed_path = "../Walking Subtask/LOG_FS/Walking_RandomSeed_FS/Results/"
s_result_randomseed_path = "../Sit-Stand Subtask/LOG/Results/"
t_result_randomseed_path = "../Turning Subtask/LOG/Results/"
f_result_randomseed_path = "../Facial Subtask/LOG_FS/Facial_RandomSeed/Results/"


# Subject ID

In [4]:
all_pid_list = []
all_pid_list_based = []
walk_pid_list, sitstand_pid_list, turn_pid_list, facial_pid_list = [], [], [], []


df_walk = pd.read_csv("../Walking Subtask/LOG_FS/Walking_RandomSeed_FS/Results/10_FS_weighted.csv")
df_sitstand = pd.read_csv("../Sit-Stand Subtask/LOG/Results/10_SitStand_combined.csv")
df_turn = pd.read_csv("../Turning Subtask/LOG/Results/10_Turning_Subtask.csv")
df_facial = pd.read_csv("../Facial Subtask/LOG_FS/Facial_RandomSeed/Results/10_FS_weighted.csv")


def get_all_pid(walk, sitstand, facial, turn):
    tmp_list = []
    
    for i in range(walk.shape[0]):
        pid = walk.loc[i, 'pid']
        tmp_list.append(pid)
    for i in range(sitstand.shape[0]):
        pid = sitstand.loc[i, 'pid']
        if (pid in tmp_list) == False:
            tmp_list.append(pid)
            # print(f'[add pid: Sit&Stand] pid: {pid}')
    for i in range(facial.shape[0]):
        pid = facial.loc[i, 'pid']
        if (pid in tmp_list) == False:
            tmp_list.append(pid)
            # print(f'[add pid: Facial] pid: {pid}')
    for i in range(turn.shape[0]):
        pid = turn.loc[i, 'pid']
        if (pid in tmp_list) == False:
            tmp_list.append(pid)
            # print(f'[add pid: Turning] pid: {pid}')
    return tmp_list

def get_all_pid_based(walk, sitstand, facial, turn, pidList):
    tmp_list = []
    pid_walk, pid_sitstand, pid_turn, pid_facial = [], [], [], []
    
    for i in range(walk.shape[0]): pid_walk.append(walk.loc[i, 'pid'])
    for i in range(sitstand.shape[0]): pid_sitstand.append(sitstand.loc[i, 'pid'])
    for i in range(turn.shape[0]): pid_turn.append(turn.loc[i, 'pid'])
    for i in range(facial.shape[0]): pid_facial.append(facial.loc[i, 'pid'])

    for pid in pidList:
        if (pid in pid_walk) and (pid in pid_sitstand) and (pid in pid_turn) and (pid in pid_facial):
            tmp_list.append(pid)
    return tmp_list, pid_walk, pid_sitstand, pid_turn, pid_facial


all_pid_list = get_all_pid(df_walk, df_sitstand, df_facial, df_turn)
all_pid_list_based, walk_pid_list, sitstand_pid_list, turn_pid_list, facial_pid_list = get_all_pid_based(df_walk, df_sitstand, df_facial, df_turn, all_pid_list)

print(f'# of valid pid: {len(all_pid_list)} subjects.')
print(all_pid_list)
print(f'\n# of valid pid (based): {len(all_pid_list_based)} subjects.')
print(all_pid_list_based)

print(f'\n# of subjects in 【Walking】 Subtask: {len(walk_pid_list)} subjects.')
print(walk_pid_list)
print(f'\n# of subjects in 【SitStand】 Subtask: {len(sitstand_pid_list)} subjects.')
print(sitstand_pid_list)
print(f'\n# of subjects in 【Turning】 Subtask: {len(turn_pid_list)} subjects.')
print(turn_pid_list)
print(f'\n# of subjects in 【Facial】 Subtask: {len(facial_pid_list)} subjects.')
print(facial_pid_list)

# of valid pid: 99 subjects.
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 48, 49, 51, 52, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 8, 14, 15, 23, 42, 47, 50, 53, 54, 55, 56]

# of valid pid (based): 81 subjects.
[1, 2, 3, 5, 6, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 48, 49, 51, 52, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

# of subjects in 【Walking】 Subtask: 88 subjects.
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 48, 49, 51, 52, 57, 

# Label & Age

In [5]:
ADLabel_dict = {}
excel_path = "../Label_Data_Info.xlsx" 

df_label = pd.read_excel(excel_path, usecols=['編號', 'Age', 'Gender', '_Label']) # [update] 2023/06/17
df_label = df_label.fillna(0)
df_label = df_label.astype({"_Label": int})

Label_arr = np.array(df_label['_Label'])
ADLabel_dict = dict(zip(df_label.編號, df_label._Label))

print(ADLabel_dict)

{1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 0, 42: 0, 43: 0, 44: 0, 45: 1, 46: 1, 47: 0, 48: 1, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 1, 60: 1, 61: 1, 62: 1, 63: 0, 64: 0, 65: 1, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0}


In [6]:
AgeLabel_dict = {}
excel_path = "../Label_Data_Info.xlsx" 

df_label = pd.read_excel(excel_path, usecols=['編號', 'Age', 'Gender'])
df_label = df_label.fillna(0)
df_label = df_label.astype({"Age": int})

Label_arr = np.array(df_label['Age'])
AgeLabel_dict = dict(zip(df_label.編號, df_label.Age))

print(AgeLabel_dict)

{1: 40, 2: 28, 3: 29, 4: 71, 5: 75, 6: 80, 7: 40, 8: 58, 9: 87, 10: 74, 11: 74, 12: 74, 13: 75, 14: 47, 15: 54, 16: 78, 17: 86, 18: 86, 19: 75, 20: 76, 21: 74, 22: 52, 23: 88, 24: 78, 25: 78, 26: 77, 27: 79, 28: 84, 29: 60, 30: 67, 31: 85, 32: 76, 33: 67, 34: 69, 35: 80, 36: 87, 37: 86, 38: 67, 39: 73, 40: 74, 41: 0, 42: 60, 43: 72, 44: 96, 45: 89, 46: 87, 47: 66, 48: 87, 49: 78, 50: 59, 51: 70, 52: 75, 53: 49, 54: 58, 55: 28, 56: 38, 57: 66, 58: 73, 59: 89, 60: 89, 61: 80, 62: 75, 63: 66, 64: 68, 65: 72, 66: 63, 67: 69, 68: 72, 69: 49, 70: 48, 71: 35, 72: 60, 73: 57, 74: 33, 75: 23, 76: 62, 77: 68, 78: 69, 79: 68, 80: 65, 81: 69, 82: 65, 83: 73, 84: 68, 85: 70, 86: 69, 87: 73, 88: 80, 89: 58, 90: 43, 91: 28, 92: 60, 93: 36, 94: 23, 95: 62, 96: 69, 97: 64, 98: 72, 99: 65, 100: 68}


# Fold Index Mapping

In [7]:
testPid_walk_5f, testPid_sitstand_5f, testPid_turn_5f, testPid_facial_5f = [], [], [], []

def get_subtask_5f_subjectId():
    walk_test_csv = '../../5_Gait_and_Eye/Training_Fix_5Fold/walk_test_5fold.csv'
    sitstand_test_csv = '../../5_Gait_and_Eye/Training_Fix_5Fold/sitstand_test_5fold.csv'
    turn_test_csv = '../../5_Gait_and_Eye/Training_Fix_5Fold/turn_test_5fold.csv'
    facial_test_csv = '../../5_Gait_and_Eye/Training_Fix_5Fold/eyes_test_5fold.csv'
    
    tmp_walk_testID_5f, tmp_sitstand_testID_5f, tmp_turn_testID_5f, tmp_facial_testID_5f = [], [], [], []
    
    walk_file = open(walk_test_csv, "r")
    tmp_walk_testID_5f = list(csv.reader(walk_file, delimiter=","))
    walk_file.close()

    sitstand_file = open(sitstand_test_csv, "r")
    tmp_sitstand_testID_5f = list(csv.reader(sitstand_file, delimiter=","))
    sitstand_file.close()

    turn_file = open(turn_test_csv, "r")
    tmp_turn_testID_5f = list(csv.reader(turn_file, delimiter=","))
    turn_file.close()

    facial_file = open(facial_test_csv, "r")
    tmp_facial_testID_5f = list(csv.reader(facial_file, delimiter=","))
    facial_file.close()
    return tmp_walk_testID_5f, tmp_sitstand_testID_5f, tmp_turn_testID_5f, tmp_facial_testID_5f


testPid_walk_5f, testPid_sitstand_5f, testPid_turn_5f, testPid_facial_5f = get_subtask_5f_subjectId()

In [8]:
walk_subjects_foldIdx, sitstand_subjects_foldIdx, turn_subject_foldIdx, facial_subjects_foldIdx = {}, {}, {}, {}

def get_each_subtask_subject():
    tmp_walk, tmp_sitstand, tmp_turn, tmp_facial = {}, {}, {}, {}
    for fold_idx in range(n_FOLD):
        tmp_list = testPid_walk_5f[fold_idx]
        for pid in tmp_list:
            tmp_walk[int(pid)] = fold_idx

        tmp_list = testPid_sitstand_5f[fold_idx]
        for pid in tmp_list:
            tmp_sitstand[int(pid)] = fold_idx

        tmp_list = testPid_turn_5f[fold_idx]
        for pid in tmp_list:
            tmp_turn[int(pid)] = fold_idx

        tmp_list = testPid_facial_5f[fold_idx]
        for pid in tmp_list:
            tmp_facial[int(pid)] = fold_idx
    return tmp_walk, tmp_sitstand, tmp_turn, tmp_facial

walk_subjects_foldIdx, sitstand_subjects_foldIdx, turn_subjects_foldIdx, facial_subjects_foldIdx = get_each_subtask_subject()

print(f'Walking Subtask: {len(walk_subjects_foldIdx)} subjects.')
print(f'SitStand Subtask: {len(sitstand_subjects_foldIdx)} subjects.')
print(f'Turn Subtask: {len(turn_subjects_foldIdx)} subjects.')
print(f'Facial Subtask: {len(facial_subjects_foldIdx)} subjects.')

Walking Subtask: 88 subjects.
SitStand Subtask: 88 subjects.
Turn Subtask: 83 subjects.
Facial Subtask: 96 subjects.


In [9]:
walk_subjects_foldIdx[1]

0

# Threshold

In [10]:
threshold_walk = {}
threshold_sitstand = {}
threshold_turn = {}
threshold_facial = {}

def read_threshold_from_CSV_file(folder):
    tmp_dict = {}
    for seed_value in random_seed_list:
        filename = folder + str(seed_value) + "/Parameters/threshold_5fold.csv"

        threshold_list = []
        with open(filename, newline='') as csvfile:
            rows = csv.reader(csvfile)
            threshold_list_string = next(rows)
        for num in threshold_list_string:
            threshold_list.append(float(num))
        tmp_dict[seed_value] = threshold_list
    return tmp_dict


threshold_walk = read_threshold_from_CSV_file("../Walking Subtask/LOG_FS/Walking_RandomSeed_FS/")
threshold_sitstand = read_threshold_from_CSV_file("../Sit-Stand Subtask/LOG/")
threshold_turn = read_threshold_from_CSV_file("../Turning Subtask/LOG/")
threshold_facial = read_threshold_from_CSV_file("../Facial Subtask/LOG_FS/Facial_RandomSeed/")
threshold_sitstand

{10: [0.4, 0.4, 0.42, 0.4, 0.4],
 20: [0.4, 0.4, 0.4, 0.445, 0.405],
 42: [0.435, 0.4, 0.4, 0.425, 0.4],
 64: [0.4, 0.4, 0.4, 0.42, 0.4],
 100: [0.41, 0.4, 0.4, 0.4, 0.4],
 123: [0.4, 0.4, 0.42, 0.4, 0.4],
 200: [0.405, 0.4, 0.42, 0.4, 0.4],
 456: [0.415, 0.405, 0.4, 0.425, 0.4],
 1001: [0.415, 0.43, 0.445, 0.4, 0.4],
 12321: [0.48, 0.4, 0.415, 0.4, 0.4]}

# Global Function

In [11]:
def create_walking_sitstand_facial_subtask_dictionary_RandomSeed(filename):
    data = pd.read_csv(filename)
    tmp_dict = {}
    for i in range(data.shape[0]):
        pid = data.loc[i, 'pid']
        p_class1 = data.loc[i, 'avgProb'] ### probability for class 1 (AD)
        prediction = data.loc[i, 'prediction']
        actual = data.loc[i, 'actual']
        
        p_class0 = 1 - p_class1 ### probability for class 0 (NC)
        
        tmp_dict[pid] = {}
        tmp_dict[pid]['prob_class0'] = p_class0
        tmp_dict[pid]['prob_class1'] = p_class1
        tmp_dict[pid]['prediction'] = prediction
        tmp_dict[pid]['actual'] = actual
    return tmp_dict

def create_turning_subtask_dictionary_RandomSeed(filename):
    data = pd.read_csv(filename)
    tmp_dict = {}
    for i in range(data.shape[0]):
        pid = data.loc[i, 'pid']
        p_class0 = data.loc[i, 'prob_NC']
        p_class1 = data.loc[i, 'prob_AD']
        prediction = data.loc[i, 'prediction']
        actual = data.loc[i, 'actual']
        
        tmp_dict[pid] = {}
        tmp_dict[pid]['prob_class0'] = p_class0
        tmp_dict[pid]['prob_class1'] = p_class1
        tmp_dict[pid]['prediction'] = prediction
        tmp_dict[pid]['actual'] = actual
    return tmp_dict

In [12]:
def calculate_acc_sen_spe_pre_f1score(actual_list, pred_list):
    actual_label_array = np.array(actual_list)
    predict_label_array = np.array(pred_list)

    ### Confusion Matrix
    matrix = confusion_matrix(actual_label_array, predict_label_array)
    TN, FP, FN, TP = matrix.ravel()

    value_accuracy = round((TP+TN)/(TP+TN+FP+FN), 4)
    value_sensitivity = round(TP/(TP+FN), 4)
    value_specificity = round(TN/(TN+FP), 4)
    value_precision = round(TP/(TP+FP), 4)
    value_f1_score = round(2*value_precision*value_sensitivity/(value_precision+value_sensitivity), 2)

    accuracy = round(value_accuracy*100, 2)
    sensitivity = round(value_sensitivity*100, 2)
    specificity = round(value_specificity*100, 2)
    precision = round(value_precision*100, 2)

    return accuracy, sensitivity, specificity, precision, value_f1_score

In [13]:
def organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_dict, s_dict, t_dict, f_dict):
    lambda_walk = w_f1score
    lambda_sitstand = s_f1score
    lambda_turn = t_f1score
    lambda_facial = f_f1score

    tmp_dict = {}
    for pid in pidList:
        tmp_dict[pid] = {}
        tmp_dict[pid]['Walk'] = {}
        tmp_dict[pid]['SitStand'] = {}
        tmp_dict[pid]['Turn'] = {}
        tmp_dict[pid]['Facial'] = {}

        if pid in walk_subjects_foldIdx:
            tmp_dict[pid]['Walk']['Fold'] = walk_subjects_foldIdx[pid]
            tmp_dict[pid]['Walk']['Threshold'] = threshold_walk[seed_value][walk_subjects_foldIdx[pid]]
            tmp_dict[pid]['Walk']['probAD'] = w_dict[pid]['prob_class1']
            tmp_dict[pid]['Walk']['conf'] = abs(threshold_walk[seed_value][walk_subjects_foldIdx[pid]]-w_dict[pid]['prob_class1'])
            tmp_dict[pid]['Walk']['probAD_weighted'] = round(w_dict[pid]['prob_class1']*lambda_walk, 4)
            
        if pid in sitstand_subjects_foldIdx:
            tmp_dict[pid]['SitStand']['Fold'] = sitstand_subjects_foldIdx[pid]
            tmp_dict[pid]['SitStand']['Threshold'] = threshold_sitstand[seed_value][sitstand_subjects_foldIdx[pid]]
            tmp_dict[pid]['SitStand']['probAD'] = s_dict[pid]['prob_class1']
            tmp_dict[pid]['SitStand']['conf'] = abs(threshold_sitstand[seed_value][sitstand_subjects_foldIdx[pid]]-s_dict[pid]['prob_class1'])
            tmp_dict[pid]['SitStand']['probAD_weighted'] = round(s_dict[pid]['prob_class1']*lambda_sitstand, 4)
            
        if pid in turn_subjects_foldIdx:
            tmp_dict[pid]['Turn']['Fold'] = turn_subjects_foldIdx[pid]
            tmp_dict[pid]['Turn']['Threshold'] = threshold_turn[seed_value][turn_subjects_foldIdx[pid]]
            tmp_dict[pid]['Turn']['probAD'] = t_dict[pid]['prob_class1']
            tmp_dict[pid]['Turn']['conf'] = abs(threshold_turn[seed_value][turn_subjects_foldIdx[pid]]-t_dict[pid]['prob_class1'])
            tmp_dict[pid]['Turn']['probAD_weighted'] = round(t_dict[pid]['prob_class1']*lambda_turn, 4)
            
        if pid in facial_subjects_foldIdx:
            tmp_dict[pid]['Facial']['Fold'] = facial_subjects_foldIdx[pid]
            tmp_dict[pid]['Facial']['Threshold'] = threshold_facial[seed_value][facial_subjects_foldIdx[pid]]
            tmp_dict[pid]['Facial']['probAD'] = f_dict[pid]['prob_class1']
            tmp_dict[pid]['Facial']['conf'] = abs(threshold_facial[seed_value][facial_subjects_foldIdx[pid]]-f_dict[pid]['prob_class1'])
            tmp_dict[pid]['Facial']['probAD_weighted'] = round(f_dict[pid]['prob_class1']*lambda_facial, 4)
    return tmp_dict

# 1 Subtask

## Walking Subtask

In [14]:
single_w_acc = []
single_w_sen = []
single_w_spe = []
single_w_pre = []
single_w_f1score = []


def analyze_single_w_result(pidList, seed_value_list):
    tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score = [], [], [], [], []
    for idx, seed_value in enumerate(seed_value_list):
        filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(filename)

        pred_list = []
        actual_list = []
        for pid in w_result_dict:
            if pid in pidList:
                pred = w_result_dict[pid]['prediction']
                actual = w_result_dict[pid]['actual']
                pred_list.append(pred)
                actual_list.append(actual)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(actual_list, pred_list)
        tmp_acc.append(acc)
        tmp_sen.append(sen)
        tmp_spe.append(spe)
        tmp_pre.append(pre)
        tmp_f1score.append(f1score)

        print(f'seed: {seed_value}, number of subjects: {len(pred_list)}')

    return tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score

single_w_acc, single_w_sen, single_w_spe, single_w_pre, single_w_f1score = analyze_single_w_result(walk_pid_list, random_seed_list)

seed: 10, number of subjects: 88
seed: 20, number of subjects: 88
seed: 42, number of subjects: 88
seed: 64, number of subjects: 88
seed: 100, number of subjects: 88
seed: 123, number of subjects: 88
seed: 200, number of subjects: 88
seed: 456, number of subjects: 88
seed: 1001, number of subjects: 88
seed: 12321, number of subjects: 88


In [15]:
print(f'Accuracy    ({len(single_w_acc)}), mean({round(statistics.mean(single_w_acc), 2)}), std({round(statistics.pstdev(single_w_acc), 2)}): {single_w_acc}')
print(f'Sensitivity ({len(single_w_sen)}), mean({round(statistics.mean(single_w_sen), 2)}), std({round(statistics.pstdev(single_w_sen), 2)}): {single_w_sen}')
print(f'Specificity ({len(single_w_spe)}), mean({round(statistics.mean(single_w_spe), 2)}), std({round(statistics.pstdev(single_w_spe), 2)}): {single_w_spe}')
print(f'Precision   ({len(single_w_pre)}), mean({round(statistics.mean(single_w_pre), 2)}), std({round(statistics.pstdev(single_w_pre), 2)}): {single_w_pre}')
print(f'F1-Score    ({len(single_w_f1score)}), mean({round(statistics.mean(single_w_f1score), 2)}), std({round(statistics.pstdev(single_w_f1score), 2)}): {single_w_f1score}')

Accuracy    (10), mean(91.25), std(1.61): [90.91, 93.18, 88.64, 90.91, 94.32, 92.05, 89.77, 90.91, 89.77, 92.05]
Sensitivity (10), mean(97.67), std(2.13): [96.67, 100.0, 96.67, 100.0, 100.0, 96.67, 93.33, 100.0, 96.67, 96.67]
Specificity (10), mean(87.93), std(2.04): [87.93, 89.66, 84.48, 86.21, 91.38, 89.66, 87.93, 86.21, 86.21, 89.66]
Precision   (10), mean(80.79), std(2.69): [80.56, 83.33, 76.32, 78.95, 85.71, 82.86, 80.0, 78.95, 78.38, 82.86]
F1-Score    (10), mean(0.88), std(0.02): [0.88, 0.91, 0.85, 0.88, 0.92, 0.89, 0.86, 0.88, 0.87, 0.89]


## Sit-Stand Subtask

In [16]:
single_s_acc = []
single_s_sen = []
single_s_spe = []
single_s_pre = []
single_s_f1score = []


def analyze_single_s_result(pidList, seed_value_list):
    tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score = [], [], [], [], []
    for idx, seed_value in enumerate(seed_value_list):
        filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv "
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(filename)

        pred_list = []
        actual_list = []
        for pid in s_result_dict:
            if pid in pidList:
                pred = s_result_dict[pid]['prediction']
                actual = s_result_dict[pid]['actual']
                pred_list.append(pred)
                actual_list.append(actual)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(actual_list, pred_list)
        tmp_acc.append(acc)
        tmp_sen.append(sen)
        tmp_spe.append(spe)
        tmp_pre.append(pre)
        tmp_f1score.append(f1score)

        print(f'seed: {seed_value}, number of subjects: {len(pred_list)}')

    return tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score

single_s_acc, single_s_sen, single_s_spe, single_s_pre, single_s_f1score = analyze_single_s_result(sitstand_pid_list, random_seed_list)

seed: 10, number of subjects: 88
seed: 20, number of subjects: 88
seed: 42, number of subjects: 88
seed: 64, number of subjects: 88
seed: 100, number of subjects: 88
seed: 123, number of subjects: 88
seed: 200, number of subjects: 88
seed: 456, number of subjects: 88
seed: 1001, number of subjects: 88
seed: 12321, number of subjects: 88


In [17]:
print(f'Accuracy    ({len(single_s_acc)}), mean({round(statistics.mean(single_s_acc), 2)}), std({round(statistics.pstdev(single_s_acc), 2)}): {single_s_acc}')
print(f'Sensitivity ({len(single_s_sen)}), mean({round(statistics.mean(single_s_sen), 2)}), std({round(statistics.pstdev(single_s_sen), 2)}): {single_s_sen}')
print(f'Specificity ({len(single_s_spe)}), mean({round(statistics.mean(single_s_spe), 2)}), std({round(statistics.pstdev(single_s_spe), 2)}): {single_s_spe}')
print(f'Precision   ({len(single_s_pre)}), mean({round(statistics.mean(single_s_pre), 2)}), std({round(statistics.pstdev(single_s_pre), 2)}): {single_s_pre}')
print(f'F1-Score    ({len(single_s_f1score)}), mean({round(statistics.mean(single_s_f1score), 2)}), std({round(statistics.pstdev(single_s_f1score), 2)}): {single_s_f1score}')

Accuracy    (10), mean(65.23), std(3.88): [65.91, 63.64, 57.95, 71.59, 65.91, 70.45, 65.91, 62.5, 61.36, 67.05]
Sensitivity (10), mean(34.67), std(11.66): [63.33, 36.67, 30.0, 43.33, 16.67, 36.67, 33.33, 30.0, 26.67, 30.0]
Specificity (10), mean(81.04), std(7.03): [67.24, 77.59, 72.41, 86.21, 91.38, 87.93, 82.76, 79.31, 79.31, 86.21]
Precision   (10), mean(49.06), std(7.93): [50.0, 45.83, 36.0, 61.9, 50.0, 61.11, 50.0, 42.86, 40.0, 52.94]
F1-Score    (10), mean(0.4), std(0.09): [0.56, 0.41, 0.33, 0.51, 0.25, 0.46, 0.4, 0.35, 0.32, 0.38]


## Turning Subtask

In [18]:
single_t_acc = []
single_t_sen = []
single_t_spe = []
single_t_pre = []
single_t_f1score = []


def analyze_single_t_result(pidList, seed_value_list):
    tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score = [], [], [], [], []
    for idx, seed_value in enumerate(seed_value_list):
        filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(filename)

        pred_list = []
        actual_list = []
        for pid in t_result_dict:
            if pid in pidList:
                pred = t_result_dict[pid]['prediction']
                actual = t_result_dict[pid]['actual']
                pred_list.append(pred)
                actual_list.append(actual)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(actual_list, pred_list)
        tmp_acc.append(acc)
        tmp_sen.append(sen)
        tmp_spe.append(spe)
        tmp_pre.append(pre)
        tmp_f1score.append(f1score)
        print(f'seed: {seed_value}, number of subjects: {len(pred_list)}')
    return tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score

single_t_acc, single_t_sen, single_t_spe, single_t_pre, single_t_f1score = analyze_single_t_result(turn_pid_list, random_seed_list)

seed: 10, number of subjects: 83
seed: 20, number of subjects: 83
seed: 42, number of subjects: 83
seed: 64, number of subjects: 83
seed: 100, number of subjects: 83
seed: 123, number of subjects: 83
seed: 200, number of subjects: 83
seed: 456, number of subjects: 83
seed: 1001, number of subjects: 83
seed: 12321, number of subjects: 83


In [19]:
print(f'Accuracy    ({len(single_t_acc)}), mean({round(statistics.mean(single_t_acc), 2)}), std({round(statistics.pstdev(single_t_acc), 2)}): {single_t_acc}')
print(f'Sensitivity ({len(single_t_sen)}), mean({round(statistics.mean(single_t_sen), 2)}), std({round(statistics.pstdev(single_t_sen), 2)}): {single_t_sen}')
print(f'Specificity ({len(single_t_spe)}), mean({round(statistics.mean(single_t_spe), 2)}), std({round(statistics.pstdev(single_t_spe), 2)}): {single_t_spe}')
print(f'Precision   ({len(single_t_pre)}), mean({round(statistics.mean(single_t_pre), 2)}), std({round(statistics.pstdev(single_t_pre), 2)}): {single_t_pre}')
print(f'F1-Score    ({len(single_t_f1score)}), mean({round(statistics.mean(single_t_f1score), 2)}), std({round(statistics.pstdev(single_t_f1score), 2)}): {single_t_f1score}')

Accuracy    (10), mean(81.81), std(0.36): [81.93, 81.93, 81.93, 80.72, 81.93, 81.93, 81.93, 81.93, 81.93, 81.93]
Sensitivity (10), mean(68.84), std(1.16): [69.23, 69.23, 69.23, 65.38, 69.23, 69.23, 69.23, 69.23, 69.23, 69.23]
Specificity (10), mean(87.72), std(0.0): [87.72, 87.72, 87.72, 87.72, 87.72, 87.72, 87.72, 87.72, 87.72, 87.72]
Precision   (10), mean(71.88), std(0.35): [72.0, 72.0, 72.0, 70.83, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0]
F1-Score    (10), mean(0.71), std(0.01): [0.71, 0.71, 0.71, 0.68, 0.71, 0.71, 0.71, 0.71, 0.71, 0.71]


## Facial Subtask

In [20]:
single_f_acc = []
single_f_sen = []
single_f_spe = []
single_f_pre = []
single_f_f1score = []

def analyze_single_f_result(pidList, seed_value_list):
    tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score = [], [], [], [], []
    for idx, seed_value in enumerate(seed_value_list):
        filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(filename)

        pred_list = []
        actual_list = []
        for pid in f_result_dict:
            if pid in pidList:
                pred = f_result_dict[pid]['prediction']
                actual = f_result_dict[pid]['actual']
                pred_list.append(pred)
                actual_list.append(actual)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(actual_list, pred_list)
        tmp_acc.append(acc)
        tmp_sen.append(sen)
        tmp_spe.append(spe)
        tmp_pre.append(pre)
        tmp_f1score.append(f1score)
        print(f'seed: {seed_value}, number of subjects: {len(pred_list)}')
    return tmp_acc, tmp_sen, tmp_spe, tmp_pre, tmp_f1score

single_f_acc, single_f_sen, single_f_spe, single_f_pre, single_f_f1score = analyze_single_f_result(facial_pid_list, random_seed_list)

seed: 10, number of subjects: 96
seed: 20, number of subjects: 96
seed: 42, number of subjects: 96
seed: 64, number of subjects: 96
seed: 100, number of subjects: 96
seed: 123, number of subjects: 96
seed: 200, number of subjects: 96
seed: 456, number of subjects: 96
seed: 1001, number of subjects: 96
seed: 12321, number of subjects: 96


In [21]:
print(f'Accuracy    ({len(single_f_acc)}), mean({round(statistics.mean(single_f_acc), 2)}), std({round(statistics.pstdev(single_f_acc), 2)}): {single_f_acc}')
print(f'Sensitivity ({len(single_f_sen)}), mean({round(statistics.mean(single_f_sen), 2)}), std({round(statistics.pstdev(single_f_sen), 2)}): {single_f_sen}')
print(f'Specificity ({len(single_f_spe)}), mean({round(statistics.mean(single_f_spe), 2)}), std({round(statistics.pstdev(single_f_spe), 2)}): {single_f_spe}')
print(f'Precision   ({len(single_f_pre)}), mean({round(statistics.mean(single_f_pre), 2)}), std({round(statistics.pstdev(single_f_pre), 2)}): {single_f_pre}')
print(f'F1-Score    ({len(single_f_f1score)}), mean({round(statistics.mean(single_f_f1score), 2)}), std({round(statistics.pstdev(single_f_f1score), 2)}): {single_f_f1score}')

Accuracy    (10), mean(80.62), std(3.93): [81.25, 73.96, 88.54, 78.12, 80.21, 76.04, 82.29, 79.17, 83.33, 83.33]
Sensitivity (10), mean(50.0), std(8.34): [55.17, 41.38, 62.07, 37.93, 48.28, 44.83, 44.83, 44.83, 58.62, 62.07]
Specificity (10), mean(93.88), std(3.43): [92.54, 88.06, 100.0, 95.52, 94.03, 89.55, 98.51, 94.03, 94.03, 92.54]
Precision   (10), mean(78.61), std(10.99): [76.19, 60.0, 100.0, 78.57, 77.78, 65.0, 92.86, 76.47, 80.95, 78.26]
F1-Score    (10), mean(0.61), std(0.08): [0.64, 0.49, 0.77, 0.51, 0.6, 0.53, 0.6, 0.57, 0.68, 0.69]


# 2 Subtask

In [22]:
def get_two_subtask_combination():
    s1 = "walk"
    s2 = "sitstand"
    s3 = "turn"
    s4 = "facial"

    combination = []
    combination.append([s1, s2])
    combination.append([s1, s3])
    combination.append([s1, s4])
    combination.append([s2, s3])
    combination.append([s2, s4])
    combination.append([s3, s4])
    return combination
    
two_subtask_combination = get_two_subtask_combination()

print(f'-------------- [Two Subtasks] --------------')
print(two_subtask_combination)

-------------- [Two Subtasks] --------------
[['walk', 'sitstand'], ['walk', 'turn'], ['walk', 'facial'], ['sitstand', 'turn'], ['sitstand', 'facial'], ['turn', 'facial']]


In [23]:
WS_acc, WS_sen, WS_spe, WS_pre, WS_f1s = [], [], [], [], []
WT_acc, WT_sen, WT_spe, WT_pre, WT_f1s = [], [], [], [], []
WF_acc, WF_sen, WF_spe, WF_pre, WF_f1s = [], [], [], [], []
ST_acc, ST_sen, ST_spe, ST_pre, ST_f1s = [], [], [], [], []
SF_acc, SF_sen, SF_spe, SF_pre, SF_f1s = [], [], [], [], []
TF_acc, TF_sen, TF_spe, TF_pre, TF_f1s = [], [], [], [], []


def get_weighted_result_two_subtask_RandomSeed(pidList, task1, task2, score1, score2, result1, result2, subtask_importance):
    lambda1 = score1
    lambda2 = score2

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        conf1, conf2 = 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            if task1=="walk": conf1 = subtask_importance[pid]['Walk']['conf']
            if task1=="sitstand": conf1 = subtask_importance[pid]['SitStand']['conf']
            if task1=="turn": conf1 = subtask_importance[pid]['Turn']['conf']
            if task1=="facial": conf1 = subtask_importance[pid]['Facial']['conf']
            
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            if task2=="walk": conf2 = subtask_importance[pid]['Walk']['conf']
            if task2=="sitstand": conf2 = subtask_importance[pid]['SitStand']['conf']
            if task2=="turn": conf2 = subtask_importance[pid]['Turn']['conf']
            if task2=="facial": conf2 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [{task1}, {task2}] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict
   


def analyze_two_subtask_result(pidList, combination, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]

        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)

        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)
        
        for comb in combination:
            task1, task2 = comb[0], comb[1]

            if task1=="walk": score1 = w_f1score; result1 = w_result_dict
            elif task1=="sitstand": score1 = s_f1score; result1 = s_result_dict
            elif task1=="turn": score1 = t_f1score; result1 = t_result_dict
            elif task1=="facial": score1 = f_f1score; result1 = f_result_dict
                
            if task2=="walk": score2 = w_f1score; result2 = w_result_dict
            elif task2=="sitstand": score2 = s_f1score; result2 = s_result_dict
            elif task2=="turn": score2 = t_f1score; result2 = t_result_dict
            elif task2=="facial": score2 = f_f1score; result2 = f_result_dict

            combined_pidList = []
            for pid in pidList:
                if pid in result1 or pid in result2:
                    combined_pidList.append(pid)

            print(f'-> Seed_value=[{seed_value}], Combined [{task1}, {task2}], Number of Subjects: {len(combined_pidList)}')
            pred_list, true_list, weighted_result_dict = get_weighted_result_two_subtask_RandomSeed(combined_pidList, task1, task2, score1, score2, result1, result2, subtask_importance)

            acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
            if task1=="walk" and task2=="sitstand":
                WS_acc.append(acc)
                WS_sen.append(sen)
                WS_spe.append(spe)
                WS_pre.append(pre)
                WS_f1s.append(f1score)
            elif task1=="walk" and task2=="turn":
                WT_acc.append(acc)
                WT_sen.append(sen)
                WT_spe.append(spe)
                WT_pre.append(pre)
                WT_f1s.append(f1score)
            elif task1=="walk" and task2=="facial":
                WF_acc.append(acc)
                WF_sen.append(sen)
                WF_spe.append(spe)
                WF_pre.append(pre)
                WF_f1s.append(f1score)
            elif task1=="sitstand" and task2=="turn":
                ST_acc.append(acc)
                ST_sen.append(sen)
                ST_spe.append(spe)
                ST_pre.append(pre)
                ST_f1s.append(f1score)
            elif task1=="sitstand" and task2=="facial":
                SF_acc.append(acc)
                SF_sen.append(sen)
                SF_spe.append(spe)
                SF_pre.append(pre)
                SF_f1s.append(f1score)
            elif task1=="turn" and task2=="facial":
                TF_acc.append(acc)
                TF_sen.append(sen)
                TF_spe.append(spe)
                TF_pre.append(pre)
                TF_f1s.append(f1score)

analyze_two_subtask_result(all_pid_list, two_subtask_combination, random_seed_list)

-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[10], Combined [walk, turn], Number of Subjects: 88
-> Seed_value=[10], Combined [walk, facial], Number of Subjects: 99
-> Seed_value=[10], Combined [sitstand, turn], Number of Subjects: 88
-> Seed_value=[10], Combined [sitstand, facial], Number of Subjects: 99
-> Seed_value=[10], Combined [turn, facial], Number of Subjects: 98
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[20], Combined [walk, turn], Number of Subjects: 88
-> Seed_value=[20], Combined [walk, facial], Number of Subjects: 99
-> Seed_value=[20], Combined [sitstand, turn], Number of Subjects: 88
-> Seed_value=[20], Combined [sitstand, facial], Number of Subjects: 99
-> Seed_value=[20], Combined [turn, facial], Number of Subjects: 98
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[42], Combined [walk, turn], Number of Subjects: 88
-> Seed_value=[42], Combined [walk, 

In [24]:
print(f'Accuracy    ({len(WS_acc)}), mean({round(statistics.mean(WS_acc), 2)}), std({round(statistics.pstdev(WS_acc), 2)}): {WS_acc}')
print(f'Sensitivity ({len(WS_sen)}), mean({round(statistics.mean(WS_sen), 2)}), std({round(statistics.pstdev(WS_sen), 2)}): {WS_sen}')
print(f'Specificity ({len(WS_spe)}), mean({round(statistics.mean(WS_spe), 2)}), std({round(statistics.pstdev(WS_spe), 2)}): {WS_spe}')
print(f'Precision   ({len(WS_pre)}), mean({round(statistics.mean(WS_pre), 2)}), std({round(statistics.pstdev(WS_pre), 2)}): {WS_pre}')
print(f'F1-Score    ({len(WS_f1s)}), mean({round(statistics.mean(WS_f1s), 2)}), std({round(statistics.pstdev(WS_f1s), 2)}): {WS_f1s}')

Accuracy    (10), mean(92.96), std(2.08): [92.05, 94.32, 89.77, 95.45, 96.59, 93.18, 90.91, 92.05, 90.91, 94.32]
Sensitivity (10), mean(97.67), std(2.13): [93.33, 100.0, 96.67, 100.0, 100.0, 96.67, 96.67, 100.0, 96.67, 96.67]
Specificity (10), mean(90.52), std(2.7): [91.38, 91.38, 86.21, 93.1, 94.83, 91.38, 87.93, 87.93, 87.93, 93.1]
Precision   (10), mean(84.35), std(3.86): [84.85, 85.71, 78.38, 88.24, 90.91, 85.29, 80.56, 81.08, 80.56, 87.88]
F1-Score    (10), mean(0.91), std(0.03): [0.89, 0.92, 0.87, 0.94, 0.95, 0.91, 0.88, 0.9, 0.88, 0.92]


In [25]:
print(f'Accuracy    ({len(WT_acc)}), mean({round(statistics.mean(WT_acc), 2)}), std({round(statistics.pstdev(WT_acc), 2)}): {WT_acc}')
print(f'Sensitivity ({len(WT_sen)}), mean({round(statistics.mean(WT_sen), 2)}), std({round(statistics.pstdev(WT_sen), 2)}): {WT_sen}')
print(f'Specificity ({len(WT_spe)}), mean({round(statistics.mean(WT_spe), 2)}), std({round(statistics.pstdev(WT_spe), 2)}): {WT_spe}')
print(f'Precision   ({len(WT_pre)}), mean({round(statistics.mean(WT_pre), 2)}), std({round(statistics.pstdev(WT_pre), 2)}): {WT_pre}')
print(f'F1-Score    ({len(WT_f1s)}), mean({round(statistics.mean(WT_f1s), 2)}), std({round(statistics.pstdev(WT_f1s), 2)}): {WT_f1s}')

Accuracy    (10), mean(92.84), std(1.02): [92.05, 93.18, 93.18, 92.05, 94.32, 94.32, 90.91, 93.18, 92.05, 93.18]
Sensitivity (10), mean(93.0), std(1.8): [90.0, 93.33, 93.33, 93.33, 93.33, 96.67, 93.33, 93.33, 93.33, 90.0]
Specificity (10), mean(92.76), std(1.5): [93.1, 93.1, 93.1, 91.38, 94.83, 93.1, 89.66, 93.1, 91.38, 94.83]
Precision   (10), mean(86.98), std(2.29): [87.1, 87.5, 87.5, 84.85, 90.32, 87.88, 82.35, 87.5, 84.85, 90.0]
F1-Score    (10), mean(0.9), std(0.01): [0.89, 0.9, 0.9, 0.89, 0.92, 0.92, 0.87, 0.9, 0.89, 0.9]


In [26]:
print(f'Accuracy    ({len(WF_acc)}), mean({round(statistics.mean(WF_acc), 2)}), std({round(statistics.pstdev(WF_acc), 2)}): {WF_acc}')
print(f'Sensitivity ({len(WF_sen)}), mean({round(statistics.mean(WF_sen), 2)}), std({round(statistics.pstdev(WF_sen), 2)}): {WF_sen}')
print(f'Specificity ({len(WF_spe)}), mean({round(statistics.mean(WF_spe), 2)}), std({round(statistics.pstdev(WF_spe), 2)}): {WF_spe}')
print(f'Precision   ({len(WF_pre)}), mean({round(statistics.mean(WF_pre), 2)}), std({round(statistics.pstdev(WF_pre), 2)}): {WF_pre}')
print(f'F1-Score    ({len(WF_f1s)}), mean({round(statistics.mean(WF_f1s), 2)}), std({round(statistics.pstdev(WF_f1s), 2)}): {WF_f1s}')

Accuracy    (10), mean(92.83), std(1.59): [91.92, 92.93, 91.92, 95.96, 94.95, 90.91, 90.91, 93.94, 91.92, 92.93]
Sensitivity (10), mean(91.29), std(3.24): [90.32, 90.32, 87.1, 100.0, 90.32, 90.32, 90.32, 90.32, 93.55, 90.32]
Specificity (10), mean(93.53), std(1.88): [92.65, 94.12, 94.12, 94.12, 97.06, 91.18, 91.18, 95.59, 91.18, 94.12]
Precision   (10), mean(86.67), std(3.44): [84.85, 87.5, 87.1, 88.57, 93.33, 82.35, 82.35, 90.32, 82.86, 87.5]
F1-Score    (10), mean(0.89), std(0.02): [0.87, 0.89, 0.87, 0.94, 0.92, 0.86, 0.86, 0.9, 0.88, 0.89]


In [27]:
print(f'Accuracy    ({len(ST_acc)}), mean({round(statistics.mean(ST_acc), 2)}), std({round(statistics.pstdev(ST_acc), 2)}): {ST_acc}')
print(f'Sensitivity ({len(ST_sen)}), mean({round(statistics.mean(ST_sen), 2)}), std({round(statistics.pstdev(ST_sen), 2)}): {ST_sen}')
print(f'Specificity ({len(ST_spe)}), mean({round(statistics.mean(ST_spe), 2)}), std({round(statistics.pstdev(ST_spe), 2)}): {ST_spe}')
print(f'Precision   ({len(ST_pre)}), mean({round(statistics.mean(ST_pre), 2)}), std({round(statistics.pstdev(ST_pre), 2)}): {ST_pre}')
print(f'F1-Score    ({len(ST_f1s)}), mean({round(statistics.mean(ST_f1s), 2)}), std({round(statistics.pstdev(ST_f1s), 2)}): {ST_f1s}')

Accuracy    (10), mean(75.45), std(2.6): [80.68, 75.0, 73.86, 76.14, 71.59, 76.14, 77.27, 75.0, 77.27, 71.59]
Sensitivity (10), mean(47.67), std(6.67): [63.33, 40.0, 46.67, 50.0, 40.0, 50.0, 50.0, 46.67, 50.0, 40.0]
Specificity (10), mean(89.83), std(1.63): [89.66, 93.1, 87.93, 89.66, 87.93, 89.66, 91.38, 89.66, 91.38, 87.93]
Precision   (10), mean(70.68), std(4.64): [76.0, 75.0, 66.67, 71.43, 63.16, 71.43, 75.0, 70.0, 75.0, 63.16]
F1-Score    (10), mean(0.57), std(0.06): [0.69, 0.52, 0.55, 0.59, 0.49, 0.59, 0.6, 0.56, 0.6, 0.49]


In [28]:
print(f'Accuracy    ({len(SF_acc)}), mean({round(statistics.mean(SF_acc), 2)}), std({round(statistics.pstdev(SF_acc), 2)}): {SF_acc}')
print(f'Sensitivity ({len(SF_sen)}), mean({round(statistics.mean(SF_sen), 2)}), std({round(statistics.pstdev(SF_sen), 2)}): {SF_sen}')
print(f'Specificity ({len(SF_spe)}), mean({round(statistics.mean(SF_spe), 2)}), std({round(statistics.pstdev(SF_spe), 2)}): {SF_spe}')
print(f'Precision   ({len(SF_pre)}), mean({round(statistics.mean(SF_pre), 2)}), std({round(statistics.pstdev(SF_pre), 2)}): {SF_pre}')
print(f'F1-Score    ({len(SF_f1s)}), mean({round(statistics.mean(SF_f1s), 2)}), std({round(statistics.pstdev(SF_f1s), 2)}): {SF_f1s}')

Accuracy    (10), mean(73.94), std(1.55): [75.76, 73.74, 76.77, 73.74, 75.76, 72.73, 72.73, 71.72, 73.74, 72.73]
Sensitivity (10), mean(22.9), std(6.03): [38.71, 22.58, 25.81, 19.35, 22.58, 19.35, 16.13, 19.35, 25.81, 19.35]
Specificity (10), mean(97.21), std(2.13): [92.65, 97.06, 100.0, 98.53, 100.0, 97.06, 98.53, 95.59, 95.59, 97.06]
Precision   (10), mean(80.68), std(11.01): [70.59, 77.78, 100.0, 85.71, 100.0, 75.0, 83.33, 66.67, 72.73, 75.0]
F1-Score    (10), mean(0.35), std(0.06): [0.5, 0.35, 0.41, 0.32, 0.37, 0.31, 0.27, 0.3, 0.38, 0.31]


In [29]:
print(f'Accuracy    ({len(TF_acc)}), mean({round(statistics.mean(TF_acc), 2)}), std({round(statistics.pstdev(TF_acc), 2)}): {TF_acc}')
print(f'Sensitivity ({len(TF_sen)}), mean({round(statistics.mean(TF_sen), 2)}), std({round(statistics.pstdev(TF_sen), 2)}): {TF_sen}')
print(f'Specificity ({len(TF_spe)}), mean({round(statistics.mean(TF_spe), 2)}), std({round(statistics.pstdev(TF_spe), 2)}): {TF_spe}')
print(f'Precision   ({len(TF_pre)}), mean({round(statistics.mean(TF_pre), 2)}), std({round(statistics.pstdev(TF_pre), 2)}): {TF_pre}')
print(f'F1-Score    ({len(TF_f1s)}), mean({round(statistics.mean(TF_f1s), 2)}), std({round(statistics.pstdev(TF_f1s), 2)}): {TF_f1s}')

Accuracy    (10), mean(80.92), std(2.09): [80.61, 83.67, 83.67, 80.61, 81.63, 77.55, 82.65, 79.59, 81.63, 77.55]
Sensitivity (10), mean(49.67), std(4.33): [50.0, 53.33, 50.0, 46.67, 46.67, 46.67, 50.0, 50.0, 60.0, 43.33]
Specificity (10), mean(94.71), std(2.56): [94.12, 97.06, 98.53, 95.59, 97.06, 91.18, 97.06, 92.65, 91.18, 92.65]
Precision   (10), mean(81.19), std(7.72): [78.95, 88.89, 93.75, 82.35, 87.5, 70.0, 88.24, 75.0, 75.0, 72.22]
F1-Score    (10), mean(0.62), std(0.04): [0.61, 0.67, 0.65, 0.6, 0.61, 0.56, 0.64, 0.6, 0.67, 0.54]


# 3 Subtask

In [30]:
def get_three_subtask_combination():
    s1 = "walk"
    s2 = "sitstand"
    s3 = "turn"
    s4 = "facial"

    combination = []
    combination.append([s1, s2, s3])
    combination.append([s1, s2, s4])
    combination.append([s1, s3, s4])
    combination.append([s2, s3, s4])
    return combination
    
Three_subtask_combination = get_three_subtask_combination()

print(f'-------------- [Three Subtasks] --------------')
print(Three_subtask_combination)

-------------- [Three Subtasks] --------------
[['walk', 'sitstand', 'turn'], ['walk', 'sitstand', 'facial'], ['walk', 'turn', 'facial'], ['sitstand', 'turn', 'facial']]


In [31]:
WST_acc, WST_sen, WST_spe, WST_pre, WST_f1s = [], [], [], [], []
WSF_acc, WSF_sen, WSF_spe, WSF_pre, WSF_f1s = [], [], [], [], []
WTF_acc, WTF_sen, WTF_spe, WTF_pre, WTF_f1s = [], [], [], [], []
STF_acc, STF_sen, STF_spe, STF_pre, STF_f1s = [], [], [], [], []


def get_weighted_result_three_subtask_RandomSeed(pidList, task1, task2, task3, score1, score2, score3, result1, result2, result3, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        conf1, conf2, conf3 = 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            if task1=="walk": conf1 = subtask_importance[pid]['Walk']['conf']
            if task1=="sitstand": conf1 = subtask_importance[pid]['SitStand']['conf']
            if task1=="turn": conf1 = subtask_importance[pid]['Turn']['conf']
            if task1=="facial": conf1 = subtask_importance[pid]['Facial']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            if task2=="walk": conf2 = subtask_importance[pid]['Walk']['conf']
            if task2=="sitstand": conf2 = subtask_importance[pid]['SitStand']['conf']
            if task2=="turn": conf2 = subtask_importance[pid]['Turn']['conf']
            if task2=="facial": conf2 = subtask_importance[pid]['Facial']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            if task3=="walk": conf3 = subtask_importance[pid]['Walk']['conf']
            if task3=="sitstand": conf3 = subtask_importance[pid]['SitStand']['conf']
            if task3=="turn": conf3 = subtask_importance[pid]['Turn']['conf']
            if task3=="facial": conf3 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [{task1}, {task2}, {task3}] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_three_subtask_result(pidList, combination, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]

        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)

        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)
        
        for comb in combination:
            task1, task2, task3 = comb[0], comb[1], comb[2]

            if task1=="walk": score1 = w_f1score; result1 = w_result_dict
            elif task1=="sitstand": score1 = s_f1score; result1 = s_result_dict
            elif task1=="turn": score1 = t_f1score; result1 = t_result_dict
            elif task1=="facial": score1 = f_f1score; result1 = f_result_dict
                
            if task2=="walk": score2 = w_f1score; result2 = w_result_dict
            elif task2=="sitstand": score2 = s_f1score; result2 = s_result_dict
            elif task2=="turn": score2 = t_f1score; result2 = t_result_dict
            elif task2=="facial": score2 = f_f1score; result2 = f_result_dict

            if task3=="walk": score3 = w_f1score; result3 = w_result_dict
            elif task3=="sitstand": score3 = s_f1score; result3 = s_result_dict
            elif task3=="turn": score3 = t_f1score; result3 = t_result_dict
            elif task3=="facial": score3 = f_f1score; result3 = f_result_dict

            combined_pidList = []
            for pid in pidList:
                if pid in result1 or pid in result2 or pid in result3:
                    combined_pidList.append(pid)

            print(f'-> Seed_value=[{seed_value}], Combined [{task1}, {task2}], Number of Subjects: {len(combined_pidList)}')
            pred_list, true_list, weighted_result_dict = get_weighted_result_three_subtask_RandomSeed(combined_pidList, task1, task2, task3, score1, score2, score3, result1, result2, result3, subtask_importance)

            acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
            if task1=="walk" and task2=="sitstand" and task3=="turn":
                WST_acc.append(acc)
                WST_sen.append(sen)
                WST_spe.append(spe)
                WST_pre.append(pre)
                WST_f1s.append(f1score)
            elif task1=="walk" and task2=="sitstand" and task3=="facial":
                WSF_acc.append(acc)
                WSF_sen.append(sen)
                WSF_spe.append(spe)
                WSF_pre.append(pre)
                WSF_f1s.append(f1score)
            elif task1=="walk" and task2=="turn" and task3=="facial":
                WTF_acc.append(acc)
                WTF_sen.append(sen)
                WTF_spe.append(spe)
                WTF_pre.append(pre)
                WTF_f1s.append(f1score)
            elif task1=="sitstand" and task2=="turn" and task3=="facial":
                STF_acc.append(acc)
                STF_sen.append(sen)
                STF_spe.append(spe)
                STF_pre.append(pre)
                STF_f1s.append(f1score)

analyze_three_subtask_result(all_pid_list, Three_subtask_combination, random_seed_list)

-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 99
-> Seed_value=[10], Combined [walk, turn], Number of Subjects: 99
-> Seed_value=[10], Combined [sitstand, turn], Number of Subjects: 99
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 99
-> Seed_value=[20], Combined [walk, turn], Number of Subjects: 99
-> Seed_value=[20], Combined [sitstand, turn], Number of Subjects: 99
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 99
-> Seed_value=[42], Combined [walk, turn], Number of Subjects: 99
-> Seed_value=[42], Combined [sitstand, turn], Number of Subjects: 99
-> Seed_value=[64], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[64], Combined [walk, sitstand], Number of Subjects: 99
-> Seed_value=[64], Combined [wa

In [32]:
print(f'Accuracy    ({len(WST_acc)}), mean({round(statistics.mean(WST_acc), 2)}), std({round(statistics.pstdev(WST_acc), 2)}): {WST_acc}')
print(f'Sensitivity ({len(WST_sen)}), mean({round(statistics.mean(WST_sen), 2)}), std({round(statistics.pstdev(WST_sen), 2)}): {WST_sen}')
print(f'Specificity ({len(WST_spe)}), mean({round(statistics.mean(WST_spe), 2)}), std({round(statistics.pstdev(WST_spe), 2)}): {WST_spe}')
print(f'Precision   ({len(WST_pre)}), mean({round(statistics.mean(WST_pre), 2)}), std({round(statistics.pstdev(WST_pre), 2)}): {WST_pre}')
print(f'F1-Score    ({len(WST_f1s)}), mean({round(statistics.mean(WST_f1s), 2)}), std({round(statistics.pstdev(WST_f1s), 2)}): {WST_f1s}')

Accuracy    (10), mean(92.39), std(1.69): [92.05, 94.32, 93.18, 89.77, 94.32, 94.32, 89.77, 92.05, 90.91, 93.18]
Sensitivity (10), mean(91.33), std(2.21): [90.0, 93.33, 90.0, 90.0, 93.33, 96.67, 90.0, 90.0, 90.0, 90.0]
Specificity (10), mean(92.93), std(1.96): [93.1, 94.83, 94.83, 89.66, 94.83, 93.1, 89.66, 93.1, 91.38, 94.83]
Precision   (10), mean(87.07), std(3.18): [87.1, 90.32, 90.0, 81.82, 90.32, 87.88, 81.82, 87.1, 84.38, 90.0]
F1-Score    (10), mean(0.89), std(0.02): [0.89, 0.92, 0.9, 0.86, 0.92, 0.92, 0.86, 0.89, 0.87, 0.9]


In [33]:
print(f'Accuracy    ({len(WSF_acc)}), mean({round(statistics.mean(WSF_acc), 2)}), std({round(statistics.pstdev(WSF_acc), 2)}): {WSF_acc}')
print(f'Sensitivity ({len(WSF_sen)}), mean({round(statistics.mean(WSF_sen), 2)}), std({round(statistics.pstdev(WSF_sen), 2)}): {WSF_sen}')
print(f'Specificity ({len(WSF_spe)}), mean({round(statistics.mean(WSF_spe), 2)}), std({round(statistics.pstdev(WSF_spe), 2)}): {WSF_spe}')
print(f'Precision   ({len(WSF_pre)}), mean({round(statistics.mean(WSF_pre), 2)}), std({round(statistics.pstdev(WSF_pre), 2)}): {WSF_pre}')
print(f'F1-Score    ({len(WSF_f1s)}), mean({round(statistics.mean(WSF_f1s), 2)}), std({round(statistics.pstdev(WSF_f1s), 2)}): {WSF_f1s}')

Accuracy    (10), mean(93.03), std(1.78): [90.91, 92.93, 92.93, 95.96, 94.95, 90.91, 90.91, 94.95, 91.92, 93.94]
Sensitivity (10), mean(91.29), std(3.55): [87.1, 90.32, 87.1, 100.0, 90.32, 90.32, 90.32, 93.55, 93.55, 90.32]
Specificity (10), mean(93.83), std(2.06): [92.65, 94.12, 95.59, 94.12, 97.06, 91.18, 91.18, 95.59, 91.18, 95.59]
Precision   (10), mean(87.23), std(3.78): [84.38, 87.5, 90.0, 88.57, 93.33, 82.35, 82.35, 90.62, 82.86, 90.32]
F1-Score    (10), mean(0.89), std(0.03): [0.86, 0.89, 0.89, 0.94, 0.92, 0.86, 0.86, 0.92, 0.88, 0.9]


In [34]:
print(f'Accuracy    ({len(WTF_acc)}), mean({round(statistics.mean(WTF_acc), 2)}), std({round(statistics.pstdev(WTF_acc), 2)}): {WTF_acc}')
print(f'Sensitivity ({len(WTF_sen)}), mean({round(statistics.mean(WTF_sen), 2)}), std({round(statistics.pstdev(WTF_sen), 2)}): {WTF_sen}')
print(f'Specificity ({len(WTF_spe)}), mean({round(statistics.mean(WTF_spe), 2)}), std({round(statistics.pstdev(WTF_spe), 2)}): {WTF_spe}')
print(f'Precision   ({len(WTF_pre)}), mean({round(statistics.mean(WTF_pre), 2)}), std({round(statistics.pstdev(WTF_pre), 2)}): {WTF_pre}')
print(f'F1-Score    ({len(WTF_f1s)}), mean({round(statistics.mean(WTF_f1s), 2)}), std({round(statistics.pstdev(WTF_f1s), 2)}): {WTF_f1s}')

Accuracy    (10), mean(92.83), std(1.72): [90.91, 94.95, 93.94, 94.95, 94.95, 92.93, 90.91, 92.93, 90.91, 90.91]
Sensitivity (10), mean(89.03), std(2.96): [87.1, 90.32, 87.1, 93.55, 87.1, 93.55, 87.1, 90.32, 90.32, 83.87]
Specificity (10), mean(94.56), std(2.28): [92.65, 97.06, 97.06, 95.59, 98.53, 92.65, 92.65, 94.12, 91.18, 94.12]
Precision   (10), mean(88.4), std(4.45): [84.38, 93.33, 93.1, 90.62, 96.43, 85.29, 84.38, 87.5, 82.35, 86.67]
F1-Score    (10), mean(0.89), std(0.03): [0.86, 0.92, 0.9, 0.92, 0.92, 0.89, 0.86, 0.89, 0.86, 0.85]


In [35]:
print(f'Accuracy    ({len(STF_acc)}), mean({round(statistics.mean(STF_acc), 2)}), std({round(statistics.pstdev(STF_acc), 2)}): {STF_acc}')
print(f'Sensitivity ({len(STF_sen)}), mean({round(statistics.mean(STF_sen), 2)}), std({round(statistics.pstdev(STF_sen), 2)}): {STF_sen}')
print(f'Specificity ({len(STF_spe)}), mean({round(statistics.mean(STF_spe), 2)}), std({round(statistics.pstdev(STF_spe), 2)}): {STF_spe}')
print(f'Precision   ({len(STF_pre)}), mean({round(statistics.mean(STF_pre), 2)}), std({round(statistics.pstdev(STF_pre), 2)}): {STF_pre}')
print(f'F1-Score    ({len(STF_f1s)}), mean({round(statistics.mean(STF_f1s), 2)}), std({round(statistics.pstdev(STF_f1s), 2)}): {STF_f1s}')

Accuracy    (10), mean(79.3), std(2.08): [82.83, 79.8, 79.8, 79.8, 80.81, 76.77, 79.8, 76.77, 80.81, 75.76]
Sensitivity (10), mean(42.9), std(6.3): [54.84, 38.71, 38.71, 45.16, 41.94, 45.16, 41.94, 38.71, 51.61, 32.26]
Specificity (10), mean(95.88), std(2.26): [95.59, 98.53, 98.53, 95.59, 98.53, 91.18, 97.06, 94.12, 94.12, 95.59]
Precision   (10), mean(83.34), std(7.53): [85.0, 92.31, 92.31, 82.35, 92.86, 70.0, 86.67, 75.0, 80.0, 76.92]
F1-Score    (10), mean(0.56), std(0.06): [0.67, 0.55, 0.55, 0.58, 0.58, 0.55, 0.57, 0.51, 0.63, 0.45]


# 4 Subtask

In [36]:
WSTF_acc, WSTF_sen, WSTF_spe, WSTF_pre, WSTF_f1s = [], [], [], [], []


def get_weighted_result_four_subtask_RandomSeed(pidList, score1, score2, score3, score4, result1, result2, result3, result4, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3
    lambda4 = score4

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        task4_probNC, task4_probAD = 0, 0
        conf1, conf2, conf3, conf4 = 0, 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            conf1 = subtask_importance[pid]['Walk']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            conf2 = subtask_importance[pid]['SitStand']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            conf3 = subtask_importance[pid]['Turn']['conf']
        if pid in result4:
            n_subtask += 1
            task4_probNC = result4[pid]['prob_class0']
            task4_probAD = result4[pid]['prob_class1']
            actual_label = result4[pid]['actual']
            conf4 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [4 subtasks] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC + lambda4*conf4*task4_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD + lambda4*conf4*task4_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_four_subtask_result(pidList, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]
        
        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)
        
        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)


        print(f'-> Seed_value={seed_value}, Combined [W+S+T+F], Number of Subjects: {len(pidList)} ')
        pred_list, true_list, weighted_result_dict = get_weighted_result_four_subtask_RandomSeed(pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict, subtask_importance)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
        WSTF_acc.append(acc)
        WSTF_sen.append(sen)
        WSTF_spe.append(spe)
        WSTF_pre.append(pre)
        WSTF_f1s.append(f1score)

analyze_four_subtask_result(all_pid_list, random_seed_list)

-> Seed_value=10, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=20, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=42, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=64, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=100, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=123, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=200, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=456, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=1001, Combined [W+S+T+F], Number of Subjects: 99 
-> Seed_value=12321, Combined [W+S+T+F], Number of Subjects: 99 


In [37]:
print(f'Accuracy    ({len(WSTF_acc)}), mean({round(statistics.mean(WSTF_acc), 2)}), std({round(statistics.pstdev(WSTF_acc), 2)}): {WSTF_acc}')
print(f'Sensitivity ({len(WSTF_sen)}), mean({round(statistics.mean(WSTF_sen), 2)}), std({round(statistics.pstdev(WSTF_sen), 2)}): {WSTF_sen}')
print(f'Specificity ({len(WSTF_spe)}), mean({round(statistics.mean(WSTF_spe), 2)}), std({round(statistics.pstdev(WSTF_spe), 2)}): {WSTF_spe}')
print(f'Precision   ({len(WSTF_pre)}), mean({round(statistics.mean(WSTF_pre), 2)}), std({round(statistics.pstdev(WSTF_pre), 2)}): {WSTF_pre}')
print(f'F1-Score    ({len(WSTF_f1s)}), mean({round(statistics.mean(WSTF_f1s), 2)}), std({round(statistics.pstdev(WSTF_f1s), 2)}): {WSTF_f1s}')

Accuracy    (10), mean(92.32), std(1.76): [90.91, 94.95, 92.93, 93.94, 94.95, 91.92, 89.9, 91.92, 89.9, 91.92]
Sensitivity (10), mean(87.1), std(2.5): [87.1, 90.32, 83.87, 90.32, 87.1, 90.32, 83.87, 87.1, 87.1, 83.87]
Specificity (10), mean(94.71), std(2.3): [92.65, 97.06, 97.06, 95.59, 98.53, 92.65, 92.65, 94.12, 91.18, 95.59]
Precision   (10), mean(88.46), std(4.56): [84.38, 93.33, 92.86, 90.32, 96.43, 84.85, 83.87, 87.1, 81.82, 89.66]
F1-Score    (10), mean(0.88), std(0.03): [0.86, 0.92, 0.88, 0.9, 0.92, 0.87, 0.84, 0.87, 0.84, 0.87]


# 2 Subtask (based)

In [38]:
def get_two_subtask_combination():
    s1 = "walk"
    s2 = "sitstand"
    s3 = "turn"
    s4 = "facial"

    combination = []
    combination.append([s1, s2])
    combination.append([s1, s3])
    combination.append([s1, s4])
    combination.append([s2, s3])
    combination.append([s2, s4])
    combination.append([s3, s4])
    return combination
    
two_subtask_combination = get_two_subtask_combination()

print(f'-------------- [Two Subtasks] --------------')
print(two_subtask_combination)

-------------- [Two Subtasks] --------------
[['walk', 'sitstand'], ['walk', 'turn'], ['walk', 'facial'], ['sitstand', 'turn'], ['sitstand', 'facial'], ['turn', 'facial']]


In [39]:
WS_acc, WS_sen, WS_spe, WS_pre, WS_f1s = [], [], [], [], []
WT_acc, WT_sen, WT_spe, WT_pre, WT_f1s = [], [], [], [], []
WF_acc, WF_sen, WF_spe, WF_pre, WF_f1s = [], [], [], [], []
ST_acc, ST_sen, ST_spe, ST_pre, ST_f1s = [], [], [], [], []
SF_acc, SF_sen, SF_spe, SF_pre, SF_f1s = [], [], [], [], []
TF_acc, TF_sen, TF_spe, TF_pre, TF_f1s = [], [], [], [], []


def get_weighted_result_two_subtask_RandomSeed_based(pidList, task1, task2, score1, score2, result1, result2, subtask_importance):
    lambda1 = score1
    lambda2 = score2

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        conf1, conf2 = 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            if task1=="walk": conf1 = subtask_importance[pid]['Walk']['conf']
            if task1=="sitstand": conf1 = subtask_importance[pid]['SitStand']['conf']
            if task1=="turn": conf1 = subtask_importance[pid]['Turn']['conf']
            if task1=="facial": conf1 = subtask_importance[pid]['Facial']['conf']
            
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            if task2=="walk": conf2 = subtask_importance[pid]['Walk']['conf']
            if task2=="sitstand": conf2 = subtask_importance[pid]['SitStand']['conf']
            if task2=="turn": conf2 = subtask_importance[pid]['Turn']['conf']
            if task2=="facial": conf2 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [{task1}, {task2}] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict
   


def analyze_two_subtask_result_based(pidList, combination, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]

        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)

        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)
        
        for comb in combination:
            task1, task2 = comb[0], comb[1]

            if task1=="walk": score1 = w_f1score; result1 = w_result_dict
            elif task1=="sitstand": score1 = s_f1score; result1 = s_result_dict
            elif task1=="turn": score1 = t_f1score; result1 = t_result_dict
            elif task1=="facial": score1 = f_f1score; result1 = f_result_dict
                
            if task2=="walk": score2 = w_f1score; result2 = w_result_dict
            elif task2=="sitstand": score2 = s_f1score; result2 = s_result_dict
            elif task2=="turn": score2 = t_f1score; result2 = t_result_dict
            elif task2=="facial": score2 = f_f1score; result2 = f_result_dict

            combined_pidList = []
            for pid in pidList:
                if pid in result1 and pid in result2:
                    combined_pidList.append(pid)

            print(f'-> Seed_value=[{seed_value}], Combined [{task1}, {task2}], Number of Subjects: {len(combined_pidList)}')
            pred_list, true_list, weighted_result_dict = get_weighted_result_two_subtask_RandomSeed_based(combined_pidList, task1, task2, score1, score2, result1, result2, subtask_importance)

            acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
            if task1=="walk" and task2=="sitstand":
                WS_acc.append(acc)
                WS_sen.append(sen)
                WS_spe.append(spe)
                WS_pre.append(pre)
                WS_f1s.append(f1score)
            elif task1=="walk" and task2=="turn":
                WT_acc.append(acc)
                WT_sen.append(sen)
                WT_spe.append(spe)
                WT_pre.append(pre)
                WT_f1s.append(f1score)
            elif task1=="walk" and task2=="facial":
                WF_acc.append(acc)
                WF_sen.append(sen)
                WF_spe.append(spe)
                WF_pre.append(pre)
                WF_f1s.append(f1score)
            elif task1=="sitstand" and task2=="turn":
                ST_acc.append(acc)
                ST_sen.append(sen)
                ST_spe.append(spe)
                ST_pre.append(pre)
                ST_f1s.append(f1score)
            elif task1=="sitstand" and task2=="facial":
                SF_acc.append(acc)
                SF_sen.append(sen)
                SF_spe.append(spe)
                SF_pre.append(pre)
                SF_f1s.append(f1score)
            elif task1=="turn" and task2=="facial":
                TF_acc.append(acc)
                TF_sen.append(sen)
                TF_spe.append(spe)
                TF_pre.append(pre)
                TF_f1s.append(f1score)

analyze_two_subtask_result_based(all_pid_list, two_subtask_combination, random_seed_list)

-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[10], Combined [walk, turn], Number of Subjects: 83
-> Seed_value=[10], Combined [walk, facial], Number of Subjects: 85
-> Seed_value=[10], Combined [sitstand, turn], Number of Subjects: 83
-> Seed_value=[10], Combined [sitstand, facial], Number of Subjects: 85
-> Seed_value=[10], Combined [turn, facial], Number of Subjects: 81
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[20], Combined [walk, turn], Number of Subjects: 83
-> Seed_value=[20], Combined [walk, facial], Number of Subjects: 85
-> Seed_value=[20], Combined [sitstand, turn], Number of Subjects: 83
-> Seed_value=[20], Combined [sitstand, facial], Number of Subjects: 85
-> Seed_value=[20], Combined [turn, facial], Number of Subjects: 81
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 88
-> Seed_value=[42], Combined [walk, turn], Number of Subjects: 83
-> Seed_value=[42], Combined [walk, 

In [40]:
print(f'Accuracy    ({len(WS_acc)}), mean({round(statistics.mean(WS_acc), 2)}), std({round(statistics.pstdev(WS_acc), 2)}): {WS_acc}')
print(f'Sensitivity ({len(WS_sen)}), mean({round(statistics.mean(WS_sen), 2)}), std({round(statistics.pstdev(WS_sen), 2)}): {WS_sen}')
print(f'Specificity ({len(WS_spe)}), mean({round(statistics.mean(WS_spe), 2)}), std({round(statistics.pstdev(WS_spe), 2)}): {WS_spe}')
print(f'Precision   ({len(WS_pre)}), mean({round(statistics.mean(WS_pre), 2)}), std({round(statistics.pstdev(WS_pre), 2)}): {WS_pre}')
print(f'F1-Score    ({len(WS_f1s)}), mean({round(statistics.mean(WS_f1s), 2)}), std({round(statistics.pstdev(WS_f1s), 2)}): {WS_f1s}')

Accuracy    (10), mean(92.96), std(2.08): [92.05, 94.32, 89.77, 95.45, 96.59, 93.18, 90.91, 92.05, 90.91, 94.32]
Sensitivity (10), mean(97.67), std(2.13): [93.33, 100.0, 96.67, 100.0, 100.0, 96.67, 96.67, 100.0, 96.67, 96.67]
Specificity (10), mean(90.52), std(2.7): [91.38, 91.38, 86.21, 93.1, 94.83, 91.38, 87.93, 87.93, 87.93, 93.1]
Precision   (10), mean(84.35), std(3.86): [84.85, 85.71, 78.38, 88.24, 90.91, 85.29, 80.56, 81.08, 80.56, 87.88]
F1-Score    (10), mean(0.91), std(0.03): [0.89, 0.92, 0.87, 0.94, 0.95, 0.91, 0.88, 0.9, 0.88, 0.92]


In [41]:
print(f'Accuracy    ({len(WT_acc)}), mean({round(statistics.mean(WT_acc), 2)}), std({round(statistics.pstdev(WT_acc), 2)}): {WT_acc}')
print(f'Sensitivity ({len(WT_sen)}), mean({round(statistics.mean(WT_sen), 2)}), std({round(statistics.pstdev(WT_sen), 2)}): {WT_sen}')
print(f'Specificity ({len(WT_spe)}), mean({round(statistics.mean(WT_spe), 2)}), std({round(statistics.pstdev(WT_spe), 2)}): {WT_spe}')
print(f'Precision   ({len(WT_pre)}), mean({round(statistics.mean(WT_pre), 2)}), std({round(statistics.pstdev(WT_pre), 2)}): {WT_pre}')
print(f'F1-Score    ({len(WT_f1s)}), mean({round(statistics.mean(WT_f1s), 2)}), std({round(statistics.pstdev(WT_f1s), 2)}): {WT_f1s}')

Accuracy    (10), mean(92.41), std(1.08): [91.57, 92.77, 92.77, 91.57, 93.98, 93.98, 90.36, 92.77, 91.57, 92.77]
Sensitivity (10), mean(91.92), std(2.07): [88.46, 92.31, 92.31, 92.31, 92.31, 96.15, 92.31, 92.31, 92.31, 88.46]
Specificity (10), mean(92.63), std(1.53): [92.98, 92.98, 92.98, 91.23, 94.74, 92.98, 89.47, 92.98, 91.23, 94.74]
Precision   (10), mean(85.14), std(2.55): [85.19, 85.71, 85.71, 82.76, 88.89, 86.21, 80.0, 85.71, 82.76, 88.46]
F1-Score    (10), mean(0.88), std(0.02): [0.87, 0.89, 0.89, 0.87, 0.91, 0.91, 0.86, 0.89, 0.87, 0.88]


In [42]:
print(f'Accuracy    ({len(WF_acc)}), mean({round(statistics.mean(WF_acc), 2)}), std({round(statistics.pstdev(WF_acc), 2)}): {WF_acc}')
print(f'Sensitivity ({len(WF_sen)}), mean({round(statistics.mean(WF_sen), 2)}), std({round(statistics.pstdev(WF_sen), 2)}): {WF_sen}')
print(f'Specificity ({len(WF_spe)}), mean({round(statistics.mean(WF_spe), 2)}), std({round(statistics.pstdev(WF_spe), 2)}): {WF_spe}')
print(f'Precision   ({len(WF_pre)}), mean({round(statistics.mean(WF_pre), 2)}), std({round(statistics.pstdev(WF_pre), 2)}): {WF_pre}')
print(f'F1-Score    ({len(WF_f1s)}), mean({round(statistics.mean(WF_f1s), 2)}), std({round(statistics.pstdev(WF_f1s), 2)}): {WF_f1s}')

Accuracy    (10), mean(93.65), std(1.68): [92.94, 92.94, 91.76, 96.47, 95.29, 92.94, 90.59, 95.29, 94.12, 94.12]
Sensitivity (10), mean(93.57), std(2.67): [92.86, 92.86, 89.29, 100.0, 92.86, 92.86, 92.86, 92.86, 96.43, 92.86]
Specificity (10), mean(93.68), std(1.95): [92.98, 92.98, 92.98, 94.74, 96.49, 92.98, 89.47, 96.49, 92.98, 94.74]
Precision   (10), mean(88.03), std(3.32): [86.67, 86.67, 86.21, 90.32, 92.86, 86.67, 81.25, 92.86, 87.1, 89.66]
F1-Score    (10), mean(0.91), std(0.02): [0.9, 0.9, 0.88, 0.95, 0.93, 0.9, 0.87, 0.93, 0.92, 0.91]


In [43]:
print(f'Accuracy    ({len(ST_acc)}), mean({round(statistics.mean(ST_acc), 2)}), std({round(statistics.pstdev(ST_acc), 2)}): {ST_acc}')
print(f'Sensitivity ({len(ST_sen)}), mean({round(statistics.mean(ST_sen), 2)}), std({round(statistics.pstdev(ST_sen), 2)}): {ST_sen}')
print(f'Specificity ({len(ST_spe)}), mean({round(statistics.mean(ST_spe), 2)}), std({round(statistics.pstdev(ST_spe), 2)}): {ST_spe}')
print(f'Precision   ({len(ST_pre)}), mean({round(statistics.mean(ST_pre), 2)}), std({round(statistics.pstdev(ST_pre), 2)}): {ST_pre}')
print(f'F1-Score    ({len(ST_f1s)}), mean({round(statistics.mean(ST_f1s), 2)}), std({round(statistics.pstdev(ST_f1s), 2)}): {ST_f1s}')

Accuracy    (10), mean(77.83), std(2.17): [80.72, 78.31, 77.11, 77.11, 74.7, 79.52, 79.52, 78.31, 79.52, 73.49]
Sensitivity (10), mean(51.92), std(5.51): [61.54, 46.15, 53.85, 50.0, 46.15, 57.69, 53.85, 53.85, 53.85, 42.31]
Specificity (10), mean(89.65), std(1.65): [89.47, 92.98, 87.72, 89.47, 87.72, 89.47, 91.23, 89.47, 91.23, 87.72]
Precision   (10), mean(69.59), std(4.46): [72.73, 75.0, 66.67, 68.42, 63.16, 71.43, 73.68, 70.0, 73.68, 61.11]
F1-Score    (10), mean(0.59), std(0.05): [0.67, 0.57, 0.6, 0.58, 0.53, 0.64, 0.62, 0.61, 0.62, 0.5]


In [44]:
print(f'Accuracy    ({len(SF_acc)}), mean({round(statistics.mean(SF_acc), 2)}), std({round(statistics.pstdev(SF_acc), 2)}): {SF_acc}')
print(f'Sensitivity ({len(SF_sen)}), mean({round(statistics.mean(SF_sen), 2)}), std({round(statistics.pstdev(SF_sen), 2)}): {SF_sen}')
print(f'Specificity ({len(SF_spe)}), mean({round(statistics.mean(SF_spe), 2)}), std({round(statistics.pstdev(SF_spe), 2)}): {SF_spe}')
print(f'Precision   ({len(SF_pre)}), mean({round(statistics.mean(SF_pre), 2)}), std({round(statistics.pstdev(SF_pre), 2)}): {SF_pre}')
print(f'F1-Score    ({len(SF_f1s)}), mean({round(statistics.mean(SF_f1s), 2)}), std({round(statistics.pstdev(SF_f1s), 2)}): {SF_f1s}')

Accuracy    (10), mean(73.53), std(1.76): [75.29, 72.94, 76.47, 71.76, 75.29, 74.12, 70.59, 71.76, 74.12, 72.94]
Sensitivity (10), mean(23.57), std(6.81): [39.29, 25.0, 28.57, 14.29, 25.0, 21.43, 14.29, 21.43, 25.0, 21.43]
Specificity (10), mean(98.07), std(2.14): [92.98, 96.49, 100.0, 100.0, 100.0, 100.0, 98.25, 96.49, 98.25, 98.25]
Precision   (10), mean(87.93), std(10.65): [73.33, 77.78, 100.0, 100.0, 100.0, 100.0, 80.0, 75.0, 87.5, 85.71]
F1-Score    (10), mean(0.36), std(0.08): [0.51, 0.38, 0.44, 0.25, 0.4, 0.35, 0.24, 0.33, 0.39, 0.34]


In [45]:
print(f'Accuracy    ({len(TF_acc)}), mean({round(statistics.mean(TF_acc), 2)}), std({round(statistics.pstdev(TF_acc), 2)}): {TF_acc}')
print(f'Sensitivity ({len(TF_sen)}), mean({round(statistics.mean(TF_sen), 2)}), std({round(statistics.pstdev(TF_sen), 2)}): {TF_sen}')
print(f'Specificity ({len(TF_spe)}), mean({round(statistics.mean(TF_spe), 2)}), std({round(statistics.pstdev(TF_spe), 2)}): {TF_spe}')
print(f'Precision   ({len(TF_pre)}), mean({round(statistics.mean(TF_pre), 2)}), std({round(statistics.pstdev(TF_pre), 2)}): {TF_pre}')
print(f'F1-Score    ({len(TF_f1s)}), mean({round(statistics.mean(TF_f1s), 2)}), std({round(statistics.pstdev(TF_f1s), 2)}): {TF_f1s}')

Accuracy    (10), mean(84.2), std(2.26): [85.19, 87.65, 86.42, 83.95, 82.72, 82.72, 85.19, 83.95, 85.19, 79.01]
Sensitivity (10), mean(56.0), std(6.2): [60.0, 64.0, 56.0, 52.0, 48.0, 56.0, 56.0, 60.0, 64.0, 44.0]
Specificity (10), mean(96.78), std(1.92): [96.43, 98.21, 100.0, 98.21, 98.21, 94.64, 98.21, 94.64, 94.64, 94.64]
Precision   (10), mean(88.93), std(6.33): [88.24, 94.12, 100.0, 92.86, 92.31, 82.35, 93.33, 83.33, 84.21, 78.57]
F1-Score    (10), mean(0.68), std(0.05): [0.71, 0.76, 0.72, 0.67, 0.63, 0.67, 0.7, 0.7, 0.73, 0.56]


# 3 Subtask (based)

In [46]:
def get_three_subtask_combination():
    s1 = "walk"
    s2 = "sitstand"
    s3 = "turn"
    s4 = "facial"

    combination = []
    combination.append([s1, s2, s3])
    combination.append([s1, s2, s4])
    combination.append([s1, s3, s4])
    combination.append([s2, s3, s4])
    return combination
    
Three_subtask_combination = get_three_subtask_combination()

print(f'-------------- [Three Subtasks] --------------')
print(Three_subtask_combination)

-------------- [Three Subtasks] --------------
[['walk', 'sitstand', 'turn'], ['walk', 'sitstand', 'facial'], ['walk', 'turn', 'facial'], ['sitstand', 'turn', 'facial']]


In [47]:
WST_acc, WST_sen, WST_spe, WST_pre, WST_f1s = [], [], [], [], []
WSF_acc, WSF_sen, WSF_spe, WSF_pre, WSF_f1s = [], [], [], [], []
WTF_acc, WTF_sen, WTF_spe, WTF_pre, WTF_f1s = [], [], [], [], []
STF_acc, STF_sen, STF_spe, STF_pre, STF_f1s = [], [], [], [], []


def get_weighted_result_three_subtask_RandomSeed_based(pidList, task1, task2, task3, score1, score2, score3, result1, result2, result3, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        conf1, conf2, conf3 = 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            if task1=="walk": conf1 = subtask_importance[pid]['Walk']['conf']
            if task1=="sitstand": conf1 = subtask_importance[pid]['SitStand']['conf']
            if task1=="turn": conf1 = subtask_importance[pid]['Turn']['conf']
            if task1=="facial": conf1 = subtask_importance[pid]['Facial']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            if task2=="walk": conf2 = subtask_importance[pid]['Walk']['conf']
            if task2=="sitstand": conf2 = subtask_importance[pid]['SitStand']['conf']
            if task2=="turn": conf2 = subtask_importance[pid]['Turn']['conf']
            if task2=="facial": conf2 = subtask_importance[pid]['Facial']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            if task3=="walk": conf3 = subtask_importance[pid]['Walk']['conf']
            if task3=="sitstand": conf3 = subtask_importance[pid]['SitStand']['conf']
            if task3=="turn": conf3 = subtask_importance[pid]['Turn']['conf']
            if task3=="facial": conf3 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [{task1}, {task2}, {task3}] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_three_subtask_result(pidList, combination, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]

        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)

        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)
        
        for comb in combination:
            task1, task2, task3 = comb[0], comb[1], comb[2]

            if task1=="walk": score1 = w_f1score; result1 = w_result_dict
            elif task1=="sitstand": score1 = s_f1score; result1 = s_result_dict
            elif task1=="turn": score1 = t_f1score; result1 = t_result_dict
            elif task1=="facial": score1 = f_f1score; result1 = f_result_dict
                
            if task2=="walk": score2 = w_f1score; result2 = w_result_dict
            elif task2=="sitstand": score2 = s_f1score; result2 = s_result_dict
            elif task2=="turn": score2 = t_f1score; result2 = t_result_dict
            elif task2=="facial": score2 = f_f1score; result2 = f_result_dict

            if task3=="walk": score3 = w_f1score; result3 = w_result_dict
            elif task3=="sitstand": score3 = s_f1score; result3 = s_result_dict
            elif task3=="turn": score3 = t_f1score; result3 = t_result_dict
            elif task3=="facial": score3 = f_f1score; result3 = f_result_dict

            combined_pidList = []
            for pid in pidList:
                if pid in result1 and pid in result2 and pid in result3:
                    combined_pidList.append(pid)

            print(f'-> Seed_value=[{seed_value}], Combined [{task1}, {task2}], Number of Subjects: {len(combined_pidList)}')
            pred_list, true_list, weighted_result_dict = get_weighted_result_three_subtask_RandomSeed_based(combined_pidList, task1, task2, task3, score1, score2, score3, result1, result2, result3, subtask_importance)

            acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
            if task1=="walk" and task2=="sitstand" and task3=="turn":
                WST_acc.append(acc)
                WST_sen.append(sen)
                WST_spe.append(spe)
                WST_pre.append(pre)
                WST_f1s.append(f1score)
            elif task1=="walk" and task2=="sitstand" and task3=="facial":
                WSF_acc.append(acc)
                WSF_sen.append(sen)
                WSF_spe.append(spe)
                WSF_pre.append(pre)
                WSF_f1s.append(f1score)
            elif task1=="walk" and task2=="turn" and task3=="facial":
                WTF_acc.append(acc)
                WTF_sen.append(sen)
                WTF_spe.append(spe)
                WTF_pre.append(pre)
                WTF_f1s.append(f1score)
            elif task1=="sitstand" and task2=="turn" and task3=="facial":
                STF_acc.append(acc)
                STF_sen.append(sen)
                STF_spe.append(spe)
                STF_pre.append(pre)
                STF_f1s.append(f1score)

analyze_three_subtask_result(all_pid_list, Three_subtask_combination, random_seed_list)

-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 83
-> Seed_value=[10], Combined [walk, sitstand], Number of Subjects: 85
-> Seed_value=[10], Combined [walk, turn], Number of Subjects: 81
-> Seed_value=[10], Combined [sitstand, turn], Number of Subjects: 81
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 83
-> Seed_value=[20], Combined [walk, sitstand], Number of Subjects: 85
-> Seed_value=[20], Combined [walk, turn], Number of Subjects: 81
-> Seed_value=[20], Combined [sitstand, turn], Number of Subjects: 81
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 83
-> Seed_value=[42], Combined [walk, sitstand], Number of Subjects: 85
-> Seed_value=[42], Combined [walk, turn], Number of Subjects: 81
-> Seed_value=[42], Combined [sitstand, turn], Number of Subjects: 81
-> Seed_value=[64], Combined [walk, sitstand], Number of Subjects: 83
-> Seed_value=[64], Combined [walk, sitstand], Number of Subjects: 85
-> Seed_value=[64], Combined [wa

In [48]:
print(f'Accuracy    ({len(WST_acc)}), mean({round(statistics.mean(WST_acc), 2)}), std({round(statistics.pstdev(WST_acc), 2)}): {WST_acc}')
print(f'Sensitivity ({len(WST_sen)}), mean({round(statistics.mean(WST_sen), 2)}), std({round(statistics.pstdev(WST_sen), 2)}): {WST_sen}')
print(f'Specificity ({len(WST_spe)}), mean({round(statistics.mean(WST_spe), 2)}), std({round(statistics.pstdev(WST_spe), 2)}): {WST_spe}')
print(f'Precision   ({len(WST_pre)}), mean({round(statistics.mean(WST_pre), 2)}), std({round(statistics.pstdev(WST_pre), 2)}): {WST_pre}')
print(f'F1-Score    ({len(WST_f1s)}), mean({round(statistics.mean(WST_f1s), 2)}), std({round(statistics.pstdev(WST_f1s), 2)}): {WST_f1s}')

Accuracy    (10), mean(91.93), std(1.79): [91.57, 93.98, 92.77, 89.16, 93.98, 93.98, 89.16, 91.57, 90.36, 92.77]
Sensitivity (10), mean(90.0), std(2.55): [88.46, 92.31, 88.46, 88.46, 92.31, 96.15, 88.46, 88.46, 88.46, 88.46]
Specificity (10), mean(92.81), std(1.99): [92.98, 94.74, 94.74, 89.47, 94.74, 92.98, 89.47, 92.98, 91.23, 94.74]
Precision   (10), mean(85.2), std(3.58): [85.19, 88.89, 88.46, 79.31, 88.89, 86.21, 79.31, 85.19, 82.14, 88.46]
F1-Score    (10), mean(0.88), std(0.03): [0.87, 0.91, 0.88, 0.84, 0.91, 0.91, 0.84, 0.87, 0.85, 0.88]


In [49]:
print(f'Accuracy    ({len(WSF_acc)}), mean({round(statistics.mean(WSF_acc), 2)}), std({round(statistics.pstdev(WSF_acc), 2)}): {WSF_acc}')
print(f'Sensitivity ({len(WSF_sen)}), mean({round(statistics.mean(WSF_sen), 2)}), std({round(statistics.pstdev(WSF_sen), 2)}): {WSF_sen}')
print(f'Specificity ({len(WSF_spe)}), mean({round(statistics.mean(WSF_spe), 2)}), std({round(statistics.pstdev(WSF_spe), 2)}): {WSF_spe}')
print(f'Precision   ({len(WSF_pre)}), mean({round(statistics.mean(WSF_pre), 2)}), std({round(statistics.pstdev(WSF_pre), 2)}): {WSF_pre}')
print(f'F1-Score    ({len(WSF_f1s)}), mean({round(statistics.mean(WSF_f1s), 2)}), std({round(statistics.pstdev(WSF_f1s), 2)}): {WSF_f1s}')

Accuracy    (10), mean(93.88), std(1.88): [91.76, 92.94, 92.94, 96.47, 95.29, 92.94, 90.59, 96.47, 94.12, 95.29]
Sensitivity (10), mean(93.57), std(3.11): [89.29, 92.86, 89.29, 100.0, 92.86, 92.86, 92.86, 96.43, 96.43, 92.86]
Specificity (10), mean(94.03), std(2.11): [92.98, 92.98, 94.74, 94.74, 96.49, 92.98, 89.47, 96.49, 92.98, 96.49]
Precision   (10), mean(88.63), std(3.6): [86.21, 86.67, 89.29, 90.32, 92.86, 86.67, 81.25, 93.1, 87.1, 92.86]
F1-Score    (10), mean(0.91), std(0.03): [0.88, 0.9, 0.89, 0.95, 0.93, 0.9, 0.87, 0.95, 0.92, 0.93]


In [50]:
print(f'Accuracy    ({len(WTF_acc)}), mean({round(statistics.mean(WTF_acc), 2)}), std({round(statistics.pstdev(WTF_acc), 2)}): {WTF_acc}')
print(f'Sensitivity ({len(WTF_sen)}), mean({round(statistics.mean(WTF_sen), 2)}), std({round(statistics.pstdev(WTF_sen), 2)}): {WTF_sen}')
print(f'Specificity ({len(WTF_spe)}), mean({round(statistics.mean(WTF_spe), 2)}), std({round(statistics.pstdev(WTF_spe), 2)}): {WTF_spe}')
print(f'Precision   ({len(WTF_pre)}), mean({round(statistics.mean(WTF_pre), 2)}), std({round(statistics.pstdev(WTF_pre), 2)}): {WTF_pre}')
print(f'F1-Score    ({len(WTF_f1s)}), mean({round(statistics.mean(WTF_f1s), 2)}), std({round(statistics.pstdev(WTF_f1s), 2)}): {WTF_f1s}')

Accuracy    (10), mean(93.33), std(1.76): [91.36, 95.06, 93.83, 95.06, 95.06, 95.06, 90.12, 93.83, 92.59, 91.36]
Sensitivity (10), mean(90.0), std(3.22): [88.0, 92.0, 88.0, 92.0, 88.0, 96.0, 88.0, 92.0, 92.0, 84.0]
Specificity (10), mean(94.82), std(2.03): [92.86, 96.43, 96.43, 96.43, 98.21, 94.64, 91.07, 94.64, 92.86, 94.64]
Precision   (10), mean(88.75), std(4.02): [84.62, 92.0, 91.67, 92.0, 95.65, 88.89, 81.48, 88.46, 85.19, 87.5]
F1-Score    (10), mean(0.89), std(0.03): [0.86, 0.92, 0.9, 0.92, 0.92, 0.92, 0.85, 0.9, 0.88, 0.86]


In [51]:
print(f'Accuracy    ({len(STF_acc)}), mean({round(statistics.mean(STF_acc), 2)}), std({round(statistics.pstdev(STF_acc), 2)}): {STF_acc}')
print(f'Sensitivity ({len(STF_sen)}), mean({round(statistics.mean(STF_sen), 2)}), std({round(statistics.pstdev(STF_sen), 2)}): {STF_sen}')
print(f'Specificity ({len(STF_spe)}), mean({round(statistics.mean(STF_spe), 2)}), std({round(statistics.pstdev(STF_spe), 2)}): {STF_spe}')
print(f'Precision   ({len(STF_pre)}), mean({round(statistics.mean(STF_pre), 2)}), std({round(statistics.pstdev(STF_pre), 2)}): {STF_pre}')
print(f'F1-Score    ({len(STF_f1s)}), mean({round(statistics.mean(STF_f1s), 2)}), std({round(statistics.pstdev(STF_f1s), 2)}): {STF_f1s}')

Accuracy    (10), mean(82.84), std(1.95): [86.42, 82.72, 82.72, 82.72, 83.95, 82.72, 81.48, 81.48, 85.19, 79.01]
Sensitivity (10), mean(49.2), std(7.39): [64.0, 48.0, 44.0, 48.0, 48.0, 56.0, 44.0, 48.0, 56.0, 36.0]
Specificity (10), mean(97.86), std(1.56): [96.43, 98.21, 100.0, 98.21, 100.0, 94.64, 98.21, 96.43, 98.21, 98.21]
Precision   (10), mean(91.66), std(5.25): [88.89, 92.31, 100.0, 92.31, 100.0, 82.35, 91.67, 85.71, 93.33, 90.0]
F1-Score    (10), mean(0.64), std(0.06): [0.74, 0.63, 0.61, 0.63, 0.65, 0.67, 0.59, 0.62, 0.7, 0.51]


# 4 Subtask (based)

In [52]:
WSTF_acc, WSTF_sen, WSTF_spe, WSTF_pre, WSTF_f1s = [], [], [], [], []


def get_weighted_result_four_subtask_RandomSeed_based(pidList, score1, score2, score3, score4, result1, result2, result3, result4, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3
    lambda4 = score4

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        task4_probNC, task4_probAD = 0, 0
        conf1, conf2, conf3, conf4 = 0, 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            conf1 = subtask_importance[pid]['Walk']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            conf2 = subtask_importance[pid]['SitStand']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            conf3 = subtask_importance[pid]['Turn']['conf']
        if pid in result4:
            n_subtask += 1
            task4_probNC = result4[pid]['prob_class0']
            task4_probAD = result4[pid]['prob_class1']
            actual_label = result4[pid]['actual']
            conf4 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [4 subtasks] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC + lambda4*conf4*task4_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD + lambda4*conf4*task4_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = probNC
            tmp_result_dict[pid]['probAD'] = probAD
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_four_subtask_result(pidList, seed_value_list):
    for idx, seed_value in enumerate(seed_value_list):
        w_f1score = single_w_f1score[idx]
        s_f1score = single_s_f1score[idx]
        t_f1score = single_t_f1score[idx]
        f_f1score = single_f_f1score[idx]
        
        w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
        s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
        t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
        f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

        w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
        s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
        t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
        f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)
        
        subtask_importance = {}
        subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)


        print(f'-> Seed_value={seed_value}, Combined [W+S+T+F], Number of Subjects: {len(pidList)} ')
        pred_list, true_list, weighted_result_dict = get_weighted_result_four_subtask_RandomSeed_based(pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict, subtask_importance)

        acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
        WSTF_acc.append(acc)
        WSTF_sen.append(sen)
        WSTF_spe.append(spe)
        WSTF_pre.append(pre)
        WSTF_f1s.append(f1score)

analyze_four_subtask_result(all_pid_list_based, random_seed_list)

-> Seed_value=10, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=20, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=42, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=64, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=100, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=123, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=200, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=456, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=1001, Combined [W+S+T+F], Number of Subjects: 81 
-> Seed_value=12321, Combined [W+S+T+F], Number of Subjects: 81 


In [53]:
print(f'Accuracy    ({len(WSTF_acc)}), mean({round(statistics.mean(WSTF_acc), 2)}), std({round(statistics.pstdev(WSTF_acc), 2)}): {WSTF_acc}')
print(f'Sensitivity ({len(WSTF_sen)}), mean({round(statistics.mean(WSTF_sen), 2)}), std({round(statistics.pstdev(WSTF_sen), 2)}): {WSTF_sen}')
print(f'Specificity ({len(WSTF_spe)}), mean({round(statistics.mean(WSTF_spe), 2)}), std({round(statistics.pstdev(WSTF_spe), 2)}): {WSTF_spe}')
print(f'Precision   ({len(WSTF_pre)}), mean({round(statistics.mean(WSTF_pre), 2)}), std({round(statistics.pstdev(WSTF_pre), 2)}): {WSTF_pre}')
print(f'F1-Score    ({len(WSTF_f1s)}), mean({round(statistics.mean(WSTF_f1s), 2)}), std({round(statistics.pstdev(WSTF_f1s), 2)}): {WSTF_f1s}')

Accuracy    (10), mean(92.84), std(1.81): [91.36, 95.06, 92.59, 93.83, 95.06, 93.83, 88.89, 93.83, 91.36, 92.59]
Sensitivity (10), mean(88.0), std(3.1): [88.0, 92.0, 84.0, 88.0, 88.0, 92.0, 84.0, 92.0, 88.0, 84.0]
Specificity (10), mean(95.0), std(2.08): [92.86, 96.43, 96.43, 96.43, 98.21, 94.64, 91.07, 94.64, 92.86, 96.43]
Precision   (10), mean(88.88), std(4.21): [84.62, 92.0, 91.3, 91.67, 95.65, 88.46, 80.77, 88.46, 84.62, 91.3]
F1-Score    (10), mean(0.88), std(0.03): [0.86, 0.92, 0.87, 0.9, 0.92, 0.9, 0.82, 0.9, 0.86, 0.87]


# One Random Seed (seed_value=10)

## 4 subtask (based)

In [54]:
def get_weighted_result_four_subtask_based(pidList, score1, score2, score3, score4, result1, result2, result3, result4, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3
    lambda4 = score4

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        task4_probNC, task4_probAD = 0, 0
        conf1, conf2, conf3, conf4 = 0, 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            conf1 = subtask_importance[pid]['Walk']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            conf2 = subtask_importance[pid]['SitStand']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            conf3 = subtask_importance[pid]['Turn']['conf']
        if pid in result4:
            n_subtask += 1
            task4_probNC = result4[pid]['prob_class0']
            task4_probAD = result4[pid]['prob_class1']
            actual_label = result4[pid]['actual']
            conf4 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [4 subtasks] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC + lambda4*conf4*task4_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD + lambda4*conf4*task4_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = round(probNC, 4)
            tmp_result_dict[pid]['probAD'] = round(probAD, 4)
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_four_subtask_result_based_randomseed10(pidList, seed_value):
    idx = 0
    
    w_f1score = single_w_f1score[idx]
    s_f1score = single_s_f1score[idx]
    t_f1score = single_t_f1score[idx]
    f_f1score = single_f_f1score[idx]
    
    w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
    s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
    t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
    f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

    w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
    s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
    t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
    f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)
    
    subtask_importance = {}
    subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)

    print(f'-> Seed_value={seed_value}, Combined [W+S+T+F], Number of Subjects: {len(pidList)} ')
    pred_list, true_list, weighted_result_dict = get_weighted_result_four_subtask_based(pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict, subtask_importance)

    acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
    print(f' - Accuracy: {acc}%')
    print(f' - Sensitivity: {sen}%')
    print(f' - Specificity: {spe}%')
    print(f' - Precision: {pre}%')
    print(f' - F1-Score: {f1score}')
    return weighted_result_dict


one_random_seed_four_subtasks_based_dict = {}
one_random_seed_four_subtasks_based_dict = analyze_four_subtask_result_based_randomseed10(all_pid_list_based, 10)


-> Seed_value=10, Combined [W+S+T+F], Number of Subjects: 81 
 - Accuracy: 91.36%
 - Sensitivity: 88.0%
 - Specificity: 92.86%
 - Precision: 84.62%
 - F1-Score: 0.86


In [55]:
print(f'Number of subjects: {len(one_random_seed_four_subtasks_based_dict)}')

one_random_seed_four_subtasks_based_dict

Number of subjects: 81


{1: {'probNC': 0.1057, 'probAD': 0.1093, 'prediction': 1, 'actual': 0},
 2: {'probNC': 0.1336, 'probAD': 0.0097, 'prediction': 0, 'actual': 0},
 3: {'probNC': 0.1263, 'probAD': 0.0215, 'prediction': 0, 'actual': 0},
 5: {'probNC': 0.0576, 'probAD': 0.0303, 'prediction': 0, 'actual': 0},
 6: {'probNC': 0.0595, 'probAD': 0.1352, 'prediction': 1, 'actual': 0},
 10: {'probNC': 0.0661, 'probAD': 0.104, 'prediction': 1, 'actual': 1},
 11: {'probNC': 0.0307, 'probAD': 0.131, 'prediction': 1, 'actual': 1},
 12: {'probNC': 0.0755, 'probAD': 0.062, 'prediction': 0, 'actual': 1},
 13: {'probNC': 0.0828, 'probAD': 0.0992, 'prediction': 1, 'actual': 0},
 16: {'probNC': 0.0426, 'probAD': 0.1194, 'prediction': 1, 'actual': 1},
 17: {'probNC': 0.0239, 'probAD': 0.1134, 'prediction': 1, 'actual': 1},
 18: {'probNC': 0.0737, 'probAD': 0.1304, 'prediction': 1, 'actual': 1},
 19: {'probNC': 0.138, 'probAD': 0.0468, 'prediction': 0, 'actual': 0},
 20: {'probNC': 0.0909, 'probAD': 0.0524, 'prediction': 0, '

## 4 subtask (combined)

In [56]:
def get_weighted_result_four_subtask_combined(pidList, score1, score2, score3, score4, result1, result2, result3, result4, subtask_importance):
    lambda1 = score1
    lambda2 = score2
    lambda3 = score3
    lambda4 = score4

    tmp_result_dict = {}
    tmp_pred_list, tmp_true_list = [], []

    for pid in pidList:
        task1_probNC, task1_probAD = 0, 0
        task2_probNC, task2_probAD = 0, 0
        task3_probNC, task3_probAD = 0, 0
        task4_probNC, task4_probAD = 0, 0
        conf1, conf2, conf3, conf4 = 0, 0, 0, 0

        n_subtask = 0
        if pid in result1:
            n_subtask += 1
            task1_probNC = result1[pid]['prob_class0']
            task1_probAD = result1[pid]['prob_class1']
            actual_label = result1[pid]['actual']
            conf1 = subtask_importance[pid]['Walk']['conf']
        if pid in result2:
            n_subtask += 1
            task2_probNC = result2[pid]['prob_class0']
            task2_probAD = result2[pid]['prob_class1']
            actual_label = result2[pid]['actual']
            conf2 = subtask_importance[pid]['SitStand']['conf']
        if pid in result3:
            n_subtask += 1
            task3_probNC = result3[pid]['prob_class0']
            task3_probAD = result3[pid]['prob_class1']
            actual_label = result3[pid]['actual']
            conf3 = subtask_importance[pid]['Turn']['conf']
        if pid in result4:
            n_subtask += 1
            task4_probNC = result4[pid]['prob_class0']
            task4_probAD = result4[pid]['prob_class1']
            actual_label = result4[pid]['actual']
            conf4 = subtask_importance[pid]['Facial']['conf']

        if n_subtask==0:
            print(f'Combined [4 subtasks] pid {pid} NOT exist.')
        else:
            probNC = (lambda1*conf1*task1_probNC + lambda2*conf2*task2_probNC + lambda3*conf3*task3_probNC + lambda4*conf4*task4_probNC)/n_subtask
            probAD = (lambda1*conf1*task1_probAD + lambda2*conf2*task2_probAD + lambda3*conf3*task3_probAD + lambda4*conf4*task4_probAD)/n_subtask
            
            if probAD >= probNC: pred_label = 1
            else: pred_label = 0
    
            tmp_result_dict[pid] = {}
            tmp_result_dict[pid]['probNC'] = round(probNC, 4)
            tmp_result_dict[pid]['probAD'] = round(probAD, 4)
            tmp_result_dict[pid]['prediction'] = pred_label
            tmp_result_dict[pid]['actual'] = actual_label
            
            tmp_pred_list.append(pred_label)
            tmp_true_list.append(actual_label)
    return tmp_pred_list, tmp_true_list, tmp_result_dict


def analyze_four_subtask_result_combined_randomseed10(pidList, seed_value):
    idx = 0
    
    w_f1score = single_w_f1score[idx]
    s_f1score = single_s_f1score[idx]
    t_f1score = single_t_f1score[idx]
    f_f1score = single_f_f1score[idx]
    
    w_filename = w_result_randomseed_path + str(seed_value) + "_FS_weighted.csv"
    s_filename = s_result_randomseed_path + str(seed_value) + "_SitStand_combined.csv"
    t_filename = t_result_randomseed_path + str(seed_value) + "_Turning_Subtask.csv "
    f_filename = f_result_randomseed_path + str(seed_value) + "_FS_weighted.csv "

    w_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(w_filename)
    s_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(s_filename)
    t_result_dict = create_turning_subtask_dictionary_RandomSeed(t_filename)
    f_result_dict = create_walking_sitstand_facial_subtask_dictionary_RandomSeed(f_filename)
    
    subtask_importance = {}
    subtask_importance = organize_subtask_importance_data_RandomSeed(seed_value, pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict)

    print(f'-> Seed_value={seed_value}, Combined [W+S+T+F], Number of Subjects: {len(pidList)} ')
    pred_list, true_list, weighted_result_dict = get_weighted_result_four_subtask_combined(pidList, w_f1score, s_f1score, t_f1score, f_f1score, w_result_dict, s_result_dict, t_result_dict, f_result_dict, subtask_importance)

    acc, sen, spe, pre, f1score = calculate_acc_sen_spe_pre_f1score(true_list, pred_list)
    print(f' - Accuracy: {acc}%')
    print(f' - Sensitivity: {sen}%')
    print(f' - Specificity: {spe}%')
    print(f' - Precision: {pre}%')
    print(f' - F1-Score: {f1score}')
    return weighted_result_dict


one_random_seed_four_subtasks_combined_dict = {}
one_random_seed_four_subtasks_combined_dict = analyze_four_subtask_result_combined_randomseed10(all_pid_list, 10)


-> Seed_value=10, Combined [W+S+T+F], Number of Subjects: 99 
 - Accuracy: 90.91%
 - Sensitivity: 87.1%
 - Specificity: 92.65%
 - Precision: 84.38%
 - F1-Score: 0.86


In [57]:
print(f'Number of subjects: {len(one_random_seed_four_subtasks_combined_dict)}')

one_random_seed_four_subtasks_combined_dict

Number of subjects: 99


{1: {'probNC': 0.1057, 'probAD': 0.1093, 'prediction': 1, 'actual': 0},
 2: {'probNC': 0.1336, 'probAD': 0.0097, 'prediction': 0, 'actual': 0},
 3: {'probNC': 0.1263, 'probAD': 0.0215, 'prediction': 0, 'actual': 0},
 4: {'probNC': 0.0396, 'probAD': 0.1861, 'prediction': 1, 'actual': 1},
 5: {'probNC': 0.0576, 'probAD': 0.0303, 'prediction': 0, 'actual': 0},
 6: {'probNC': 0.0595, 'probAD': 0.1352, 'prediction': 1, 'actual': 0},
 7: {'probNC': 0.158, 'probAD': 0.0173, 'prediction': 0, 'actual': 0},
 9: {'probNC': 0.0952, 'probAD': 0.1363, 'prediction': 1, 'actual': 1},
 10: {'probNC': 0.0661, 'probAD': 0.104, 'prediction': 1, 'actual': 1},
 11: {'probNC': 0.0307, 'probAD': 0.131, 'prediction': 1, 'actual': 1},
 12: {'probNC': 0.0755, 'probAD': 0.062, 'prediction': 0, 'actual': 1},
 13: {'probNC': 0.0828, 'probAD': 0.0992, 'prediction': 1, 'actual': 0},
 16: {'probNC': 0.0426, 'probAD': 0.1194, 'prediction': 1, 'actual': 1},
 17: {'probNC': 0.0239, 'probAD': 0.1134, 'prediction': 1, 'act